In [2]:
#set up github

In [3]:
%cd '/content/Object-Detection-Neural-Networks'
!git branch

/content/Object-Detection-Neural-Networks
* main


In [4]:
#CURRENT BRANCH
!git switch "hyperparameter_tuning"

Branch 'hyperparameter_tuning' set up to track remote branch 'hyperparameter_tuning' from 'origin'.
Switched to a new branch 'hyperparameter_tuning'


In [7]:
git_push("Add more hyperparameters to search space", "hyperparameter_tuning")

## Definition of Utility Functions: <a class="anchor" name="util_func"></a>

In [5]:
!pip install sahi -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.2/87.2 kB 2.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.2/114.2 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 MB 34.4 MB/s eta 0:00:00


In [6]:
import shutil

# !git pull
# !git status

#check remotes
#!git remote -v


def git_push(message, branch):
  #move to git repo before using git commands
  %cd '/content/Object-Detection-Neural-Networks'

  #copy current notebook to cloned git repo
  file_to_copy = '/content/drive/MyDrive/Colab Notebooks/UoL FP/my_data_model_test/UoL Final Project based on CM3015 template "deep learning on public dataset".ipynb'
  git_repo = '/content/Object-Detection-Neural-Networks'

  #overwrite the file in the local cloned github repo
  shutil.copy(file_to_copy, git_repo)

  !git add 'UoL Final Project based on CM3015 template "deep learning on public dataset".ipynb'

  !git commit -a -m "{message}"
  !git push origin "{branch}"

def publish_branch(branch):
  %cd '/content/Object-Detection-Neural-Networks'
  !git push -u origin "{branch}"

def create_branch(branch):
  %cd '/content/Object-Detection-Neural-Networks'
  !git checkout -b "{branch}"

In [7]:
import random
import numpy as np
import torch

device = torch.device("cuda:0")
print(device)

#the google style guide suggest that a docstring is not necessary for the set_seeds() function
#as it does not meet any of the following critera
#A docstring is mandatory for every function that has one or more of the following properties:
  # - being part of the public API
  # - nontrivial size
  # - non-obvious logic
#https://github.com/google/styleguide/blob/gh-pages/pyguide.md#38-comments-and-docstrings

#THE SET_SEEDS FUNCTION IS SOURCED FROM:
#https://learnopencv.com/fine-tuning-faster-r-cnn/#aioseo-code-walkthrough
def set_seeds():
  # fix random seeds
  SEED_VALUE = 42
  random.seed(SEED_VALUE)
  np.random. seed(SEED_VALUE)
  torch.manual_seed(SEED_VALUE)
  if torch.cuda.is_available:
    torch.cuda.manual_seed(SEED_VALUE)
    torch.cuda.manual_seed_all(SEED_VALUE)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True

cuda:0


In [8]:
import torch
from torch.utils.data import Dataset
from torch import nn
import numpy as np
import os
import cv2
from torchvision import transforms
import pandas as pd
import ast

class UAVImageDataset(Dataset):
    """Overrides the PyTorch Dataset class

      Defines how raw data should be read and transformed ready for model training

      Attributes:
          data_dir_imgs: file string path to raw images
          transforms: any Generalized R-CNN transforms to apply to the data
          all_imgs: list containing the filepath for each image
          all_targets: list of dictionaries - each dictionary contains a boxes and labels key and represents the
                        bounding boxes for the image at the corresponding index in all_imgs
    """

    def __init__(self,
                 data_dir_imgs: str,
                 df_path: str,
                 classes_start_at_zero: bool,
                 transform: nn.Module =None) -> None:
        """Loads image file paths and targets into class (instance) attributes

          Targets are loaded from .csv file (a pandas data frame that was converted to .csv)
          Img file paths and img targets are stored in separate arrays: 'self.all_imgs' and 'self.all_targets'
          Targets (boxes and labels) are compiled into a dictionary for each image
          An image file path and it's targets are stored at the same index in their respective arrays

        Args:
          data_dir_imgs: a string representing the file path to the image folder of the dataset
          df_path: a string representing the file path to the .csv file (this .csv file should be a pandas dataframe that was converted to .csv)
          classes_start_at_zero: this faciliates support for both Faster R-CNN and YOLO ; as Faster R-CNN
                                  requires that classes start at 1, whilst YOLO requires classes at index 0.
          transforms: any img transforms to be applied
        """

        self.data_dir_imgs = data_dir_imgs
        self.transforms = transforms

        file_names = os.listdir(path=data_dir_imgs)
        number_of_images = len(file_names)

        self.all_imgs = []
        self.all_targets = []

        df =  pd.read_csv(df_path)
        assert number_of_images == df.shape[0]

        for index, row in df.iterrows():
            img = data_dir_imgs + '/'+ row['Img ID']
            self.all_imgs.append(img)

            img_targets = {
                        "boxes": [],
                        "labels": []
                    }

            if pd.isna(row['Boxes']) and pd.isna(row['Labels']):
                #follow 2 lines of code sourced from:
                #https://discuss.pytorch.org/t/fasterrcnn-images-with-no-objects-present-cause-an-error/117974/13
                #to address the issue of images with no ground truth bounding boxes
                img_targets["boxes"] = torch.as_tensor(np.array(np.zeros((0, 4)), dtype=float))
                img_targets["labels"] = torch.as_tensor(np.array([], dtype=int), dtype=torch.int64)

            else:
              # append labels and boxes
                labels = row['Labels'][1:-1]
                labels = labels.split(",")
                try:
                  labels = [int(x) for x in labels]
                except:
                  print(labels)

                if not classes_start_at_zero:
                  # Faster R-CNN requires that lebls start at 1
                  img_targets['labels'] = labels
                else:
                  # YOLO requires that labels start at 0
                  img_targets['labels'] = [x-1 for x in labels]



                bboxes = ast.literal_eval(row['Boxes'])
                for bbox in bboxes:
                    x1, y1, width, height = bbox
                    img_targets['boxes'].append([x1, y1, width, height])

            self.all_targets.append(img_targets)

    def __len__(self):
        return len(self.all_imgs)

    def __getitem__(self, idx: int) -> tuple:
      """Returns image and target at the specified index

          Images are loaded and then converted to tensors (as required by PyTorch for model training)
          Raw bbox coordinates are in YOLOv8 format (x,y,w,h) and are relative to img width and height
          __getitem__ converts these coordinates to x,y,x2,y2 ready for input to Faster R-CNN

        Args:
          idx: the index of the img (and targets) to retrieve

        Returns:
            tuple of the img and targets at the specified index

        Raises: AssertionError if a bbox with negative height or width is found
        """
      img = self.all_imgs[idx]
      img_read = cv2.imread(img)
      del img
      image_transform = transforms.ToTensor()  #cv library reads as a numpy array, needs to be a pytorch tensor to be compatible
      img = image_transform(img_read)
      del image_transform

      # identify if the data point has been pre-processed
      # this accounts for validation set which is loaded multiple times
      if len(self.all_targets[idx]['labels'])!= 0:
        if isinstance(self.all_targets[idx]['boxes'][0][0], str):

          list_tensors = []
          for i in range(len(self.all_targets[idx]['boxes'])):
            x1, y1, width, height = self.all_targets[idx]['boxes'][i]
            x1, width,  = float(x1)*img_read.shape[1], float(width)*img_read.shape[1],
            y1, height = float(y1)*img_read.shape[0], float(height)*img_read.shape[0]
            x2 = x1 + width
            y2 = y1 + height

            # check for invalid bboxes
            if x2 < x1 or x1 == x2 or y2 < y1 or y1 == y2:
                raise AssertionError('Invalid Ground Truth Bounding Box')
                print(x1, y1, x2, y2)

            list_tensors.append(torch.tensor([x1-width/2, y1-height/2, x2-width/2, y2-height/2]))

          self.all_targets[idx]['boxes'] = torch.stack(list_tensors, dim=0)
          self.all_targets[idx]['labels'] = torch.as_tensor(self.all_targets[idx]['labels'])


      return img, self.all_targets[idx], self.all_imgs[idx]

In [9]:
import unittest
import torch

class TestDatasetPreprocessing(unittest.TestCase):
    """Checks that the integrity of the dataset is preserved after pre-processing

        Checks that image data is preserved correctly, that no data is lost and that
        all data is in the correct format ready for model training

        Attributes:
            dataset_classes_1_upwards: dataset that is formatted ready for Faster R-CNN (classes start at 1)
            dataset_classes_0_upwards: dataset that is formatted ready for YOLO (classes start at 0)
    """
    def setUp(self):
      # run these unit tests using a much smaller dataset
      # otherwise loading large dataset will make unit tests uncessarily computationally expensive
      # as these tests are checking the integrity of pre-processing functions
      # they are not assessing the dataset itself
      self.dataset_classes_1_upwards = UAVImageDataset(data_dir_imgs="/content/drive/MyDrive/UoL Final Project/Project Data/official project data/2 classes - image size test/train/images",
                                df_path="/content/drive/MyDrive/UoL Final Project/Project Data/official project data/2 classes - image size test/train_pd.csv",
                                classes_start_at_zero=False)

      self.dataset_classes_0_upwards = UAVImageDataset(data_dir_imgs="/content/drive/MyDrive/UoL Final Project/Project Data/official project data/2 classes - image size test/train/images",
                                df_path="/content/drive/MyDrive/UoL Final Project/Project Data/official project data/2 classes - image size test/train_pd.csv",
                                classes_start_at_zero=True)

    def test_img_conversion(self):
      """Checks that colour channels, image height and width are preserved during conversion to torch.tensor

        Args:
            None

        Returns:
            None

        Raises:
            AssertionError if converted image does not match original image
        """
      for i in range(self.dataset_classes_1_upwards.__len__()):
        img, targets, img_path = self.dataset_classes_1_upwards.__getitem__(i)
        num_colour_channels, img_height, img_width = img.size()
        # check colour channels have been preserved
        self.assertEqual(num_colour_channels, 3)
        # check image width and height have been preserved
        img_shape = cv2.imread(img_path).shape
        self.assertEqual(img_height, img_shape[0])
        self.assertEqual(img_width, img_shape[1])

    def test_number_imgs_labels(self):
      """Checks that no boxes, labels or images are lost during pre-processing

         Checks that the number of images and bboxes tensors and labels tensors is the same
         Checks that the length of bboxes tensors and label tensors are the same

        Args:
            None

        Returns:
            None

        Raises:
            AssertionError if any image, bbox or label has been lost during pre-processing
        """
      # check that there are the same number of img and target files
      self.assertEqual(len(self.dataset_classes_1_upwards.all_imgs), len(self.dataset_classes_1_upwards.all_targets))
      for i in range(self.dataset_classes_1_upwards.__len__()):
        img, targets, img_path = self.dataset_classes_1_upwards.__getitem__(i)
        boxes, labels = targets['boxes'], targets['labels']
        self.assertEqual(len(boxes), len(labels))

    def test_label_img_type(self):
      """Checks images, bboxes and labels are of type torch.Tensor as required for Faster R-CNN model training

        Args:
            None

        Returns:
            None

        Raises:
            AssertionError if data is not of correct type
        """
      for i in range(self.dataset_classes_1_upwards.__len__()):
        img, targets, img_path = self.dataset_classes_1_upwards.__getitem__(i)
        self.assertIsInstance(img, torch.Tensor)
        boxes, labels = targets['boxes'], targets['labels']
        self.assertIsInstance(boxes, torch.Tensor)
        self.assertIsInstance(labels, torch.Tensor)


    def test_label_formatting(self):
      """Checks that classes have been formatted correctly as specified by parameter 'classes_start_at_zero'

        dataset class UAVImageDataset has a paramter classes_start_at_zero which allows a dataset to be formatted
        ready for either YOLO or Faster R-CNN ; YOLO requires that classes start at 0 whilst Faster R-CNN
        requires that classes start at 1 (0 is reserved for 'background')

        Args:
            None

        Returns:
            None

        Raises:
            AssertionError if classes do not start at the specified index
        """
      for i in range(self.dataset_classes_1_upwards.__len__()):
        img, targets, img_path = self.dataset_classes_1_upwards.__getitem__(i)
        labels = targets['labels']
        self.assertNotIn(0, labels)

      classes_start_at_zero = False
      for i in range(self.dataset_classes_1_upwards.__len__()):
        img, targets, img_path = self.dataset_classes_0_upwards.__getitem__(i)
        labels = targets['labels']
        if 0 in labels:
          classes_start_at_zero = True
      self.assertTrue(classes_start_at_zero)

In [10]:
import unittest
import torch

class TestDataset(unittest.TestCase):
    """Checks the quality of the datasets used to train, test and validate model performance

        Checks for class imbalances that could lead to a biased model

        Attributes:
            train_dataset: the pre-processed images, labels and bboxes used to train the model
            val_dataset: the pre-processed images, labels and bboxes used to validate model performance during training
            test_dataset: the pre-processed images, labels and bboxes used to test the model
    """
    def setUp(self):
      self.train_dataset = UAVImageDataset(data_dir_imgs="/content/drive/MyDrive/UoL Final Project/Project Data/official project data/UAV riverine plastic - null images removed - original sizes/train/images",
                                df_path="/content/drive/MyDrive/UoL Final Project/Project Data/official project data/UAV riverine plastic - null images removed - original sizes/train_pd.csv",
                                classes_start_at_zero=False)

      self.val_dataset = UAVImageDataset(data_dir_imgs="/content/drive/MyDrive/UoL Final Project/Project Data/official project data/UAV riverine plastic - null images removed - original sizes/valid/images",
                                      df_path="/content/drive/MyDrive/UoL Final Project/Project Data/official project data/UAV riverine plastic - null images removed - original sizes/val_pd.csv",
                                    classes_start_at_zero=False)

      self.test_dataset = UAVImageDataset(data_dir_imgs="/content/drive/MyDrive/UoL Final Project/Project Data/official project data/UAV riverine plastic - null images removed - original sizes/test/images",
                                      df_path="/content/drive/MyDrive/UoL Final Project/Project Data/official project data/UAV riverine plastic - null images removed - original sizes/test_pd.csv",
                                    classes_start_at_zero=False)

      self.class_labels = {'1': 'plastic', '2': 'water hyacinth'}
      # class imbalance thresholds sourced from developers.google
      # https://developers.google.com/machine-learning/crash-course/overfitting/imbalanced-datasets
      self.MILD_CLASS_IMBALANCE = {'lower_limit': 0.2, 'upper_limit':0.4}
      self.MODERATE_CLASS_IMBALANCE = {'lower_limit': 0.01, 'upper_limit':0.2}
      self.SEVERE_CLASS_IMBALANCE = 0.01

      self.MAX_PERCENT_NULL_IMAGES = 0.1

    def test_class_balance(self):
      """Checks the quality of the dataset

        Calculates class counts and determines if there is a mild, moderate or severe class imbalance.
        Checks each data subset (train/val/test) separately to help ensure good quality data at each stage in the pipeline

        Args:
            None

        Returns:
            None

        Raises:
            AssertionError with custom message specifiying whether a class imbalance is mild, moderate or severe
            (according to the thresholds established by developers.google.com)
        """
      for idx, dataset_split in enumerate([self.train_dataset, self.val_dataset, self.test_dataset]):
        class_1_count = 0
        class_2_count = 0
        for i in range(dataset_split.__len__()):
          img, targets, img_path = dataset_split.__getitem__(i)
          bboxes, labels = targets['boxes'], targets['labels']

          for label in labels:
            current_label = label.item() #use .item() to convert from tensor to int - for easy equality check
            if current_label == 1:
              class_1_count += 1
            else:
              class_2_count += 1

        #calc percentage of data represented by minority class
        minority_class = min(class_1_count, class_2_count)
        total_num_labels = class_1_count + class_2_count
        percent_minority = minority_class/total_num_labels
        # check for imbalances
        self.assertGreater(percent_minority, self.SEVERE_CLASS_IMBALANCE, f"Severe class imbalance detected in data split: {idx} [train,val,test]")
        self.assertTrue(percent_minority > self.MODERATE_CLASS_IMBALANCE['lower_limit'] and percent_minority > self.MODERATE_CLASS_IMBALANCE['upper_limit'],
                        f"Moderate class imabalce detected in data split: {idx} [train,val,test]")
        self.assertTrue(percent_minority > self.MILD_CLASS_IMBALANCE['lower_limit'] and percent_minority > self.MILD_CLASS_IMBALANCE['upper_limit'],
                        f"Mild class imabalce detected in data split: {idx} [train,val,test]")

    def test_proportion_null_images(self):
      """Checks the number of null images as proportion of dataset

        Checks percentages of null images is between 0-10% of dataset
        as recommended by Glenn Jocher, creator of YOLOv5, YOLOv8 and founder of Ultralytics
        https://github.com/ultralytics/ultralytics/issues/7981

        Args:
            None

        Returns:
            None

        Raises:
            AssertionError with custom message if number of null images is greater than 10%
        """
      for idx, dataset_split in enumerate([self.train_dataset, self.val_dataset, self.test_dataset]):
        null_img_count = 0
        for i in range(dataset_split.__len__()):
          img, targets, img_path = dataset_split.__getitem__(i)
          bboxes, labels = targets['boxes'], targets['labels']
          if len(labels) ==0:
            null_img_count += 1

        percent_null_imgs = null_img_count / dataset_split.__len__()
        self.assertLess(percent_null_imgs, self.MAX_PERCENT_NULL_IMAGES, f"More than 10% null images detected in data split: {idx} [train,val,test]")

In [11]:
# if __name__=='__main__':
#     unittest.main(argv=[''], verbosity=2, exit=False)

In [12]:
import torchvision
def format_batch(batch: torch.utils.data.dataset.Subset ,
                 model: torchvision.models.detection) -> dict:
  """Formats batch for processing on GPU

    Gathers all images and targets into separate lists ready to feed into the model
    Also moves all images and targets to the same device as the model ready for processing

    Args:
        batch: a subset of the PyTorch dataset

        model: used to determine which device the batch data should be sent to

    Returns:
        dictionary of prepared batch data ; has 2 keys 'images and targets'

    Raises:
        None
    """

  images = []
  device = next(model.parameters()).device

  for img, targets_dict, img_file_path in batch:
      if img.get_device() != 0 and targets_dict["boxes"].get_device() != 0:
        img = img.to(device)
        targets_dict["boxes"] = targets_dict["boxes"].to(device)
        targets_dict["labels"] = targets_dict["labels"].to(device)
        images.append(img)

      # Data type conversions required by Mac GPU 'MPS'
      #format to tensor of dtype float 32 as supported by MPS
      # targets_dict['boxes'] = targets_dict['boxes'].type(torch.float32)
      # targets_dict['labels'] = targets_dict['labels'].type(torch.int64)

  return {'images': images, 'targets': [item[1] for item in batch]}

In [13]:
# Function check_bboxes is sourced from the forward() method in PyTorch source code file generalized_rcnn.py
# https://github.com/pytorch/vision/blob/f40c8df02c197d1a9e194210e40dee0e6a6cb1c3/torchvision/models/detection/generalized_rcnn.py
# The source code for this code block had the following comment:
# "TODO: Move this to a function"
# So I have acted on this comment and added a docstring to the function and type hinting
def check_bboxes(targets:list) -> None:
  """Checks that bounding boxes are of positive height and width

    Args:
        targets: a list of dictionaries ; each dictionary has 2 keys 'boxes' and 'labels'
                (keys boxes and lables both have tensor values)

    Returns:
        None

    Raises:
        Value Error if bounidng boxes have negative height or width
    """
  if targets is not None:
        for target_idx, target in enumerate(targets):
            boxes = target["boxes"]
            degenerate_boxes = boxes[:, 2:] <= boxes[:, :2]
            if degenerate_boxes.any():
                # print the first degenerate box
                bb_idx = torch.where(degenerate_boxes.any(dim=1))[0][0]
                degen_bb: List[float] = boxes[bb_idx].tolist()
                raise ValueError(
                    "All bounding boxes should have positive height and width."
                    f" Found invalid box {degen_bb} for target at index {target_idx}."
                )
            else:
              return


In [14]:
import matplotlib.pyplot as plt

def plot_loss(epochs: list, training_losses: list, val_losses: list) -> None:
  """Creates line graph of both training and validation loss after all epochs have been completed

    Implemented using matplotlib plot()

    Args:
        epochs: list containing values ranging from 1 to <num_epochs>,

        training_losses: list containing training loss for each epoch,

        val_losses: list containing validation loss for each epoch,

    Returns:
        No value returned ; only plt.show()

    Raises:
        AssertionError: Different number of training loss, validation loss and epochs
    """
  assert len(epochs) == len(training_losses) == len(val_losses), "Each argument must be the same length"

  fig, ax = plt.subplots()
  ax.set_xlabel("Epochs")
  ax.set_ylabel("Epoch loss")
  training_plot, = ax.plot(epochs, training_losses, label="Training Loss")
  val_plot, = ax.plot(epochs, val_losses, label="Validation Loss")
  ax.set_title("Training and Validation Loss")
  handles, labels = ax.get_legend_handles_labels()
  ax.legend(handles, labels)

  fig.show()

In [15]:
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import torch

def visualise_predictions(img_id: int,
                         test_dataset: torch.utils.data.dataset.Subset,
                         predicted: list) -> None:

  """Visualises model predictions (bounding boxes) on top of the corresponding image

    Retrieves the original image from the test set and plots bouning boxes onto the image.
    Each classes is represented by a different colour bounding box.

    Args:
        img_id: the number of the image within the batch,

        test_dataset: Subset of PyTorch dataset ; each item in the array is a tuple consisting of
        a tensor representing the image and a dictionary containing 'boxes' and 'labels' keys,

        precicted: array of model predictions for set of images ; the array consists of
        dictionaries each with 2 keys: boxes and labels

    Returns:
        No value returned ; only plt.show()

    Raises:
        IndexError: An error occurred accessing the image at the specified index.
    """

  try:
    img = test_dataset[img_id][0].permute(1,2,0)
    predictions = predicted[img_id]

    fig, ax = plt.subplots()
    ax.imshow(img)

    for (index, box) in enumerate(predictions['boxes']):
      #move data from GPU to CPU ready for visualisation
      cpu_box = box.cpu()
      predicted_label = predictions['labels'][index]

      #class names: ['migrating_single', 'plastic', 'water_hyacinth']
      colour_mappings = ['y', 'white', 'orange']
      #draw bounding box
      try:
        bbox = patches.Rectangle((cpu_box[0], cpu_box[1]), cpu_box[2], cpu_box[3], linewidth=1, edgecolor=colour_mappings[predicted_label], facecolor='none')
      except IndexError:
        print("Bounding box does not have 4 coordinates")
        print(cpu_box)
        raise

      # Add the patch to the Axes
      ax.add_patch(bbox)
    plt.show()

  except IndexError:
      print(f'Provided dataset is of length {len(test_dataset)} - image index {img_id} not within range')
      raise


In [16]:
def free_gpu_ram(dataloader: torch.utils.data.DataLoader) -> None:
  for batch in dataloader:
      for item in batch:
        del item
  torch.cuda.empty_cache()
  time.sleep(60) #allow time for GPU RAM to be cleared before training is 'finished'
  return

In [17]:
#no deps flag required to not overwrite NVIDIA drivers
#overriding the driver pip packages causes model crash
!pip install torchmetrics==1.6.0 --no-deps
!pip install lightning-utilities==0.11.8 --no-deps

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 926.4/926.4 kB 8.5 MB/s eta 0:00:00


In [18]:
from torchmetrics.detection.mean_ap import MeanAveragePrecision
from sahi.prediction import visualize_object_predictions
import numpy
import sahi
from sahi import AutoDetectionModel
from typing import Optional

def sahi_inference_eval(detection_model: AutoDetectionModel,
                        folder_path: str,
                        test_dataset_loader: torch.utils.data.dataloader,
                        img_export_path: str,
                        slice_h_w: Optional[tuple[int, int]] = None,
                        overlap_h_w: Optional[tuple[float, float]] = None,) -> dict[str, float]:

  """Executes Slicing Aided Hyper Inference (SAHI) for the given model

    SAHI is implemented using the sahi pip library
    Computes and returns the mAP50 and mAP75 values

  Args:
      detection_model: the model wrapped in a sahi AutoDetectionModel class
      folder_path: the path to the folder of images
      test_dataset_loader: the PyTorch dataset loader
      img_export_path: the path to the folder where resulting image predictions will be saved
      slice_h_w: the height and width of slices - this is optional as SAHI will call get_auto_slice_params if this param is not specified
      overlap_h_w: the height and width of overlap ratio - this is optional as SAHI will call get_auto_slice_params if this param is not specified

  Returns:
      A dictionary of mAP values: mAP50, mAP50-95 and class wise mAP

  Raises:
      None
  """

  img_file_names = os.listdir(path=folder_path)

  map50_metric = MeanAveragePrecision(iou_type="bbox", iou_thresholds=[0.5], class_metrics=True, extended_summary=True)
  map50_95_metric = MeanAveragePrecision(iou_type="bbox", class_metrics=True, extended_summary=True)

  all_predictions = []
  all_ground_truths = []
  i=0
  for batch in test_dataset_loader:
    for img, targets, file_path in batch:
      # get sliced predictions
      if slice_h_w and overlap_h_w:
        img_predictions = get_sliced_prediction(
          detection_model=detection_model,
          image = file_path,
          slice_height=slice_h_w[0],
          slice_width=slice_h_w[1],
          overlap_height_ratio=overlap_h_w[0],
          overlap_width_ratio=overlap_h_w[1],
          verbose=0
        )
      else:
        img_predictions = get_sliced_prediction(
          detection_model=detection_model,
          image = file_path,
          verbose=0
        )

      # visualise
      img = cv2.imread(file_path, cv2.IMREAD_UNCHANGED)
      img_converted = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
      numpy_img= numpy.asarray(img_converted)
      visualize_object_predictions(numpy_img,
                                   object_prediction_list=img_predictions.object_prediction_list,
                                   output_dir=img_export_path,
                                   file_name=f"{i}",
                                   hide_labels=1,
                                   export_format="jpg"
                                   )
      i=i+1

      # format predictions ready to calculate mAP metric
      img_predictions = img_predictions.to_coco_annotations()
      img_predictions = {
          "boxes": torch.tensor([pred['bbox'] for pred in img_predictions]),
          "scores": torch.tensor([pred['score'] for pred in img_predictions]),
          "labels": torch.tensor([pred['category_id'] for pred in img_predictions]) #needs to be int ?
      }
      # convert bboxes annotations to same format as ground truth
      for index, box in enumerate(img_predictions['boxes']):
        x,y,w,h = box
        img_predictions['boxes'][index][2] = x+w
        img_predictions['boxes'][index][3] = y+h

      all_predictions.append(img_predictions)

      # torchmetrics requires all bboxes in targets to be of type tensor
      for index, item in enumerate(targets['boxes']):
        targets['boxes'][index] = torch.tensor(list(map(lambda x: float(x), targets['boxes'][index])))
        targets['boxes'][index] = torch.tensor(targets['boxes'][index])

      all_ground_truths.append({"boxes": targets['boxes'], "labels": targets['labels']})

  # calculate mAP at two different thresholds
  map50_metric.update(all_predictions, all_ground_truths)
  map50_metric = map50_metric.compute()

  map50_95_metric.update(all_predictions, all_ground_truths)
  map50_95_metric = map50_95_metric.compute()

  return {'map50': map50_metric['map'], 'map50_95': map50_95_metric['map'], 'map_per_class': map50_95_metric['map_per_class']}

## Experiment with overrides to achieve custom loss

In [19]:
import torch.nn.functional as F

def weighted_fastrcnn_loss(class_logits, box_regression, labels, regression_targets):
    # type: (Tensor, Tensor, List[Tensor], List[Tensor]) -> Tuple[Tensor, Tensor]
    """
    Computes the loss for Faster R-CNN.

    Args:
        class_logits (Tensor)
        box_regression (Tensor)
        labels (list[BoxList])
        regression_targets (Tensor)

    Returns:
        classification_loss (Tensor)
        box_loss (Tensor)
    """

    labels = torch.cat(labels, dim=0)
    regression_targets = torch.cat(regression_targets, dim=0)

    weights = torch.tensor([1. ,3. ,1. ]).cuda() #one weight for background class
    classification_loss = F.cross_entropy(class_logits, labels, weight=weights)

    # get indices that correspond to the regression targets for
    # the corresponding ground truth labels, to be used with
    # advanced indexing
    sampled_pos_inds_subset = torch.where(labels > 0)[0]
    labels_pos = labels[sampled_pos_inds_subset]
    N, num_classes = class_logits.shape
    box_regression = box_regression.reshape(N, box_regression.size(-1) // 4, 4)

    box_loss = F.smooth_l1_loss(
        box_regression[sampled_pos_inds_subset, labels_pos],
        regression_targets[sampled_pos_inds_subset],
        beta=1 / 9,
        reduction="sum",
    )
    box_loss = box_loss / labels.numel()

    return classification_loss, box_loss

In [20]:
from torchvision.models.detection.roi_heads import RoIHeads, maskrcnn_loss, maskrcnn_inference, keypointrcnn_loss, keypointrcnn_inference


class ModifiedROIHeads(RoIHeads):
  def __init__(
        self,
        box_roi_pool,
        box_head,
        box_predictor,
        # Faster R-CNN training
        fg_iou_thresh,
        bg_iou_thresh,
        batch_size_per_image,
        positive_fraction,
        bbox_reg_weights,
        # Faster R-CNN inference
        score_thresh,
        nms_thresh,
        detections_per_img,
        # Mask
        mask_roi_pool=None,
        mask_head=None,
        mask_predictor=None,
        keypoint_roi_pool=None,
        keypoint_head=None,
        keypoint_predictor=None,
    ):
        super().__init__(
        box_roi_pool,
        box_head,
        box_predictor,
        # Faster R-CNN training
        fg_iou_thresh,
        bg_iou_thresh,
        batch_size_per_image,
        positive_fraction,
        bbox_reg_weights,
        # Faster R-CNN inference
        score_thresh,
        nms_thresh,
        detections_per_img,
        # Mask
        mask_roi_pool,
        mask_head,
        mask_predictor,
        keypoint_roi_pool,
        keypoint_head,
        keypoint_predictor)

  def forward(
        self,
        features,  # type: Dict[str, Tensor]
        proposals,  # type: List[Tensor]
        image_shapes,  # type: List[Tuple[int, int]]
        targets=None,  # type: Optional[List[Dict[str, Tensor]]]
    ):
        # type: (...) -> Tuple[List[Dict[str, Tensor]], Dict[str, Tensor]]
        """
        Args:
            features (List[Tensor])
            proposals (List[Tensor[N, 4]])
            image_shapes (List[Tuple[H, W]])
            targets (List[Dict])
        """
        if targets is not None:
            for t in targets:
                # TODO: https://github.com/pytorch/pytorch/issues/26731
                floating_point_types = (torch.float, torch.double, torch.half)
                if not t["boxes"].dtype in floating_point_types:
                    raise TypeError(f"target boxes must of float type, instead got {t['boxes'].dtype}")
                if not t["labels"].dtype == torch.int64:
                    raise TypeError(f"target labels must of int64 type, instead got {t['labels'].dtype}")
                if self.has_keypoint():
                    if not t["keypoints"].dtype == torch.float32:
                        raise TypeError(f"target keypoints must of float type, instead got {t['keypoints'].dtype}")

        if self.training:
            proposals, matched_idxs, labels, regression_targets = self.select_training_samples(proposals, targets)
        else:
            labels = None
            regression_targets = None
            matched_idxs = None

        box_features = self.box_roi_pool(features, proposals, image_shapes)
        box_features = self.box_head(box_features)
        class_logits, box_regression = self.box_predictor(box_features)

        result: List[Dict[str, torch.Tensor]] = []
        losses = {}
        if self.training:
            if labels is None:
                raise ValueError("labels cannot be None")
            if regression_targets is None:
                raise ValueError("regression_targets cannot be None")
                # USE MODIDIFIED LOSS FUNCTION HERE
            loss_classifier, loss_box_reg = weighted_fastrcnn_loss(class_logits, box_regression, labels, regression_targets)
            losses = {"loss_classifier": loss_classifier, "loss_box_reg": loss_box_reg}
        else:
            boxes, scores, labels = self.postprocess_detections(class_logits, box_regression, proposals, image_shapes)
            num_images = len(boxes)
            for i in range(num_images):
                result.append(
                    {
                        "boxes": boxes[i],
                        "labels": labels[i],
                        "scores": scores[i],
                    }
                )

        if self.has_mask():
            mask_proposals = [p["boxes"] for p in result]
            if self.training:
                if matched_idxs is None:
                    raise ValueError("if in training, matched_idxs should not be None")

                # during training, only focus on positive boxes
                num_images = len(proposals)
                mask_proposals = []
                pos_matched_idxs = []
                for img_id in range(num_images):
                    pos = torch.where(labels[img_id] > 0)[0]
                    mask_proposals.append(proposals[img_id][pos])
                    pos_matched_idxs.append(matched_idxs[img_id][pos])
            else:
                pos_matched_idxs = None

            if self.mask_roi_pool is not None:
                mask_features = self.mask_roi_pool(features, mask_proposals, image_shapes)
                mask_features = self.mask_head(mask_features)
                mask_logits = self.mask_predictor(mask_features)
            else:
                raise Exception("Expected mask_roi_pool to be not None")

            loss_mask = {}
            if self.training:
                if targets is None or pos_matched_idxs is None or mask_logits is None:
                    raise ValueError("targets, pos_matched_idxs, mask_logits cannot be None when training")

                gt_masks = [t["masks"] for t in targets]
                gt_labels = [t["labels"] for t in targets]
                rcnn_loss_mask = maskrcnn_loss(mask_logits, mask_proposals, gt_masks, gt_labels, pos_matched_idxs)
                loss_mask = {"loss_mask": rcnn_loss_mask}
            else:
                labels = [r["labels"] for r in result]
                masks_probs = maskrcnn_inference(mask_logits, labels)
                for mask_prob, r in zip(masks_probs, result):
                    r["masks"] = mask_prob

            losses.update(loss_mask)

        # keep none checks in if conditional so torchscript will conditionally
        # compile each branch
        if (
            self.keypoint_roi_pool is not None
            and self.keypoint_head is not None
            and self.keypoint_predictor is not None
        ):
            keypoint_proposals = [p["boxes"] for p in result]
            if self.training:
                # during training, only focus on positive boxes
                num_images = len(proposals)
                keypoint_proposals = []
                pos_matched_idxs = []
                if matched_idxs is None:
                    raise ValueError("if in trainning, matched_idxs should not be None")

                for img_id in range(num_images):
                    pos = torch.where(labels[img_id] > 0)[0]
                    keypoint_proposals.append(proposals[img_id][pos])
                    pos_matched_idxs.append(matched_idxs[img_id][pos])
            else:
                pos_matched_idxs = None

            keypoint_features = self.keypoint_roi_pool(features, keypoint_proposals, image_shapes)
            keypoint_features = self.keypoint_head(keypoint_features)
            keypoint_logits = self.keypoint_predictor(keypoint_features)

            loss_keypoint = {}
            if self.training:
                if targets is None or pos_matched_idxs is None:
                    raise ValueError("both targets and pos_matched_idxs should not be None when in training mode")

                gt_keypoints = [t["keypoints"] for t in targets]
                rcnn_loss_keypoint = keypointrcnn_loss(
                    keypoint_logits, keypoint_proposals, gt_keypoints, pos_matched_idxs
                )
                loss_keypoint = {"loss_keypoint": rcnn_loss_keypoint}
            else:
                if keypoint_logits is None or keypoint_proposals is None:
                    raise ValueError(
                        "both keypoint_logits and keypoint_proposals should not be None when not in training mode"
                    )

                keypoints_probs, kp_scores = keypointrcnn_inference(keypoint_logits, keypoint_proposals)
                for keypoint_prob, kps, r in zip(keypoints_probs, kp_scores, result):
                    r["keypoints"] = keypoint_prob
                    r["keypoints_scores"] = kps
            losses.update(loss_keypoint)

        return result, losses


In [21]:
from torchvision.models.detection.generalized_rcnn import GeneralizedRCNN
from torchvision.ops import MultiScaleRoIAlign
from torchvision.models.detection.faster_rcnn import _default_anchorgen, TwoMLPHead
from torchvision.models.detection.rpn import RPNHead, RegionProposalNetwork
from torchvision.models.detection.transform import GeneralizedRCNNTransform

class ModifiedFasterRCNN(GeneralizedRCNN):
    """
    Implements Faster R-CNN.

    The input to the model is expected to be a list of tensors, each of shape [C, H, W], one for each
    image, and should be in 0-1 range. Different images can have different sizes.

    The behavior of the model changes depending on if it is in training or evaluation mode.

    During training, the model expects both the input tensors and targets (list of dictionary),
    containing:
        - boxes (``FloatTensor[N, 4]``): the ground-truth boxes in ``[x1, y1, x2, y2]`` format, with
          ``0 <= x1 < x2 <= W`` and ``0 <= y1 < y2 <= H``.
        - labels (Int64Tensor[N]): the class label for each ground-truth box

    The model returns a Dict[Tensor] during training, containing the classification and regression
    losses for both the RPN and the R-CNN.

    During inference, the model requires only the input tensors, and returns the post-processed
    predictions as a List[Dict[Tensor]], one for each input image. The fields of the Dict are as
    follows:
        - boxes (``FloatTensor[N, 4]``): the predicted boxes in ``[x1, y1, x2, y2]`` format, with
          ``0 <= x1 < x2 <= W`` and ``0 <= y1 < y2 <= H``.
        - labels (Int64Tensor[N]): the predicted labels for each image
        - scores (Tensor[N]): the scores or each prediction

    Args:
        backbone (nn.Module): the network used to compute the features for the model.
            It should contain an out_channels attribute, which indicates the number of output
            channels that each feature map has (and it should be the same for all feature maps).
            The backbone should return a single Tensor or and OrderedDict[Tensor].
        num_classes (int): number of output classes of the model (including the background).
            If box_predictor is specified, num_classes should be None.
        min_size (int): Images are rescaled before feeding them to the backbone:
            we attempt to preserve the aspect ratio and scale the shorter edge
            to ``min_size``. If the resulting longer edge exceeds ``max_size``,
            then downscale so that the longer edge does not exceed ``max_size``.
            This may result in the shorter edge beeing lower than ``min_size``.
        max_size (int): See ``min_size``.
        image_mean (Tuple[float, float, float]): mean values used for input normalization.
            They are generally the mean values of the dataset on which the backbone has been trained
            on
        image_std (Tuple[float, float, float]): std values used for input normalization.
            They are generally the std values of the dataset on which the backbone has been trained on
        rpn_anchor_generator (AnchorGenerator): module that generates the anchors for a set of feature
            maps.
        rpn_head (nn.Module): module that computes the objectness and regression deltas from the RPN
        rpn_pre_nms_top_n_train (int): number of proposals to keep before applying NMS during training
        rpn_pre_nms_top_n_test (int): number of proposals to keep before applying NMS during testing
        rpn_post_nms_top_n_train (int): number of proposals to keep after applying NMS during training
        rpn_post_nms_top_n_test (int): number of proposals to keep after applying NMS during testing
        rpn_nms_thresh (float): NMS threshold used for postprocessing the RPN proposals
        rpn_fg_iou_thresh (float): minimum IoU between the anchor and the GT box so that they can be
            considered as positive during training of the RPN.
        rpn_bg_iou_thresh (float): maximum IoU between the anchor and the GT box so that they can be
            considered as negative during training of the RPN.
        rpn_batch_size_per_image (int): number of anchors that are sampled during training of the RPN
            for computing the loss
        rpn_positive_fraction (float): proportion of positive anchors in a mini-batch during training
            of the RPN
        rpn_score_thresh (float): only return proposals with an objectness score greater than rpn_score_thresh
        box_roi_pool (MultiScaleRoIAlign): the module which crops and resizes the feature maps in
            the locations indicated by the bounding boxes
        box_head (nn.Module): module that takes the cropped feature maps as input
        box_predictor (nn.Module): module that takes the output of box_head and returns the
            classification logits and box regression deltas.
        box_score_thresh (float): during inference, only return proposals with a classification score
            greater than box_score_thresh
        box_nms_thresh (float): NMS threshold for the prediction head. Used during inference
        box_detections_per_img (int): maximum number of detections per image, for all classes.
        box_fg_iou_thresh (float): minimum IoU between the proposals and the GT box so that they can be
            considered as positive during training of the classification head
        box_bg_iou_thresh (float): maximum IoU between the proposals and the GT box so that they can be
            considered as negative during training of the classification head
        box_batch_size_per_image (int): number of proposals that are sampled during training of the
            classification head
        box_positive_fraction (float): proportion of positive proposals in a mini-batch during training
            of the classification head
        bbox_reg_weights (Tuple[float, float, float, float]): weights for the encoding/decoding of the
            bounding boxes

    Example::

        >>> import torch
        >>> import torchvision
        >>> from torchvision.models.detection import FasterRCNN
        >>> from torchvision.models.detection.rpn import AnchorGenerator
        >>> # load a pre-trained model for classification and return
        >>> # only the features
        >>> backbone = torchvision.models.mobilenet_v2(weights=MobileNet_V2_Weights.DEFAULT).features
        >>> # FasterRCNN needs to know the number of
        >>> # output channels in a backbone. For mobilenet_v2, it's 1280,
        >>> # so we need to add it here
        >>> backbone.out_channels = 1280
        >>>
        >>> # let's make the RPN generate 5 x 3 anchors per spatial
        >>> # location, with 5 different sizes and 3 different aspect
        >>> # ratios. We have a Tuple[Tuple[int]] because each feature
        >>> # map could potentially have different sizes and
        >>> # aspect ratios
        >>> anchor_generator = AnchorGenerator(sizes=((32, 64, 128, 256, 512),),
        >>>                                    aspect_ratios=((0.5, 1.0, 2.0),))
        >>>
        >>> # let's define what are the feature maps that we will
        >>> # use to perform the region of interest cropping, as well as
        >>> # the size of the crop after rescaling.
        >>> # if your backbone returns a Tensor, featmap_names is expected to
        >>> # be ['0']. More generally, the backbone should return an
        >>> # OrderedDict[Tensor], and in featmap_names you can choose which
        >>> # feature maps to use.
        >>> roi_pooler = torchvision.ops.MultiScaleRoIAlign(featmap_names=['0'],
        >>>                                                 output_size=7,
        >>>                                                 sampling_ratio=2)
        >>>
        >>> # put the pieces together inside a FasterRCNN model
        >>> model = FasterRCNN(backbone,
        >>>                    num_classes=2,
        >>>                    rpn_anchor_generator=anchor_generator,
        >>>                    box_roi_pool=roi_pooler)
        >>> model.eval()
        >>> x = [torch.rand(3, 300, 400), torch.rand(3, 500, 400)]
        >>> predictions = model(x)
    """

    def __init__(
        self,
        backbone,
        num_classes=None,
        # transform parameters
        min_size=800,
        max_size=1333,
        image_mean=None,
        image_std=None,
        # RPN parameters
        rpn_anchor_generator=None,
        rpn_head=None,
        rpn_pre_nms_top_n_train=2000,
        rpn_pre_nms_top_n_test=1000,
        rpn_post_nms_top_n_train=2000,
        rpn_post_nms_top_n_test=1000,
        rpn_nms_thresh=0.7,
        rpn_fg_iou_thresh=0.7,
        rpn_bg_iou_thresh=0.3,
        rpn_batch_size_per_image=256,
        rpn_positive_fraction=0.5,
        rpn_score_thresh=0.0,
        # Box parameters
        box_roi_pool=None,
        box_head=None,
        box_predictor=None,
        box_score_thresh=0.05,
        box_nms_thresh=0.5,
        box_detections_per_img=100,
        box_fg_iou_thresh=0.5,
        box_bg_iou_thresh=0.5,
        box_batch_size_per_image=512,
        box_positive_fraction=0.25,
        bbox_reg_weights=None,
        **kwargs,
    ):

        if not hasattr(backbone, "out_channels"):
            raise ValueError(
                "backbone should contain an attribute out_channels "
                "specifying the number of output channels (assumed to be the "
                "same for all the levels)"
            )

        if not isinstance(rpn_anchor_generator, (AnchorGenerator, type(None))):
            raise TypeError(
                f"rpn_anchor_generator should be of type AnchorGenerator or None instead of {type(rpn_anchor_generator)}"
            )
        if not isinstance(box_roi_pool, (MultiScaleRoIAlign, type(None))):
            raise TypeError(
                f"box_roi_pool should be of type MultiScaleRoIAlign or None instead of {type(box_roi_pool)}"
            )

        if num_classes is not None:
            if box_predictor is not None:
                raise ValueError("num_classes should be None when box_predictor is specified")
        else:
            if box_predictor is None:
                raise ValueError("num_classes should not be None when box_predictor is not specified")

        out_channels = backbone.out_channels

        if rpn_anchor_generator is None:
            rpn_anchor_generator = _default_anchorgen()
        if rpn_head is None:
            rpn_head = RPNHead(out_channels, rpn_anchor_generator.num_anchors_per_location()[0])

        rpn_pre_nms_top_n = dict(training=rpn_pre_nms_top_n_train, testing=rpn_pre_nms_top_n_test)
        rpn_post_nms_top_n = dict(training=rpn_post_nms_top_n_train, testing=rpn_post_nms_top_n_test)

        # MODIFICATION HERE
        rpn = RegionProposalNetwork(
            rpn_anchor_generator,
            rpn_head,
            rpn_fg_iou_thresh,
            rpn_bg_iou_thresh,
            rpn_batch_size_per_image,
            rpn_positive_fraction,
            rpn_pre_nms_top_n,
            rpn_post_nms_top_n,
            rpn_nms_thresh,
            score_thresh=rpn_score_thresh,
        )

        if box_roi_pool is None:
            box_roi_pool = MultiScaleRoIAlign(featmap_names=["0", "1", "2", "3"], output_size=7, sampling_ratio=2)

        if box_head is None:
            resolution = box_roi_pool.output_size[0]
            representation_size = 1024
            box_head = TwoMLPHead(out_channels * resolution**2, representation_size)

        if box_predictor is None:
            representation_size = 1024
            box_predictor = FastRCNNPredictor(representation_size, num_classes)

        # MODIFICATION HERE
        roi_heads = ModifiedROIHeads(
            # Box
            box_roi_pool,
            box_head,
            box_predictor,
            box_fg_iou_thresh,
            box_bg_iou_thresh,
            box_batch_size_per_image,
            box_positive_fraction,
            bbox_reg_weights,
            box_score_thresh,
            box_nms_thresh,
            box_detections_per_img,
        )

        if image_mean is None:
            image_mean = [0.485, 0.456, 0.406]
        if image_std is None:
            image_std = [0.229, 0.224, 0.225]
        transform = GeneralizedRCNNTransform(min_size, max_size, image_mean, image_std, **kwargs)

        super().__init__(backbone, rpn, roi_heads, transform)

## Definition of Faster R-CNN Wrapper Class: <a class="anchor" name="faster_rcnn_class"></a>

In [22]:
# !pip install torchmetrics==1.6.1
# !pip install lightning-utilities==0.11.9

#https://lightning.ai/docs/torchmetrics/stable/detection/mean_average_precision.html
from torchmetrics.detection.mean_ap import MeanAveragePrecision
import time
from torchvision.models.detection import FasterRCNN
from typing import Tuple, List, Dict, Optional
import torch
from torch import Tensor
from torch.utils.data import Dataset, DataLoader
from collections import OrderedDict
from torchvision.models.detection.roi_heads import fastrcnn_loss
from torchvision.models.detection.rpn import concat_box_prediction_layers
import numpy as np
import datetime
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor

class FasterRCNNWrapper():
  """A wrapper class that adds to the functionality of the PyTorch FasterRCNN class

    The FasterRCNN model initialised by PyTorch FasterRCNN is stored in the 'model' attribute
    The methods of this class connect to the 'model' attribute to train and test the model

    Attributes:
        model: the object initialiased by PyTorch FasterRCNN init()
    """
  # Call the init method of FasterRCNN within the init() of this subclass
  # so that the model can be saved as an instance attribute
  # pass the default values of parameters to FasterRCNN init()
  # in case class is instantiate without specifying all values
  def __init__(self,
              backbone,
              num_classes=None,
              # transform parameters
              min_size=800,
              max_size=1333,
              image_mean=None,
              image_std=None,
              # RPN parameters
              rpn_anchor_generator=None,
              rpn_head=None,
              rpn_pre_nms_top_n_train=2000,
              rpn_pre_nms_top_n_test=1000,
              rpn_post_nms_top_n_train=2000,
              rpn_post_nms_top_n_test=1000,
              rpn_nms_thresh=0.7,
              rpn_fg_iou_thresh=0.7,
              rpn_bg_iou_thresh=0.3,
              rpn_batch_size_per_image=256,
              rpn_positive_fraction=0.5,
              rpn_score_thresh=0.0,
              # Box parameters
              box_roi_pool=None,
              box_head=None,
              box_predictor=None,
              box_score_thresh=0.05,
              box_nms_thresh=0.5,
              box_detections_per_img=100,
              box_fg_iou_thresh=0.5,
              box_bg_iou_thresh=0.5,
              box_batch_size_per_image=512,
              box_positive_fraction=0.25,
              bbox_reg_weights=None,
              **kwargs,) -> None:
    """Initialises the 'model' attribute to be a FasterRCNN model

    The 'model' attribute it utilised by all methods in this class

    Args: All parameters required by the init() of PyTorch FasterRCNN

    Returns: None

    Raies: AssertionError is model is going to be initialised as NoneType
    """

    self.model = FasterRCNN( #ModifiedFasterRCNN(
        backbone,
        num_classes,
        # transform parameters
        min_size,
        max_size,
        image_mean,
        image_std,
        # RPN parameters
        rpn_anchor_generator,
        rpn_head,
        rpn_pre_nms_top_n_train,
        rpn_pre_nms_top_n_test,
        rpn_post_nms_top_n_train,
        rpn_post_nms_top_n_test,
        rpn_nms_thresh,
        rpn_fg_iou_thresh,
        rpn_bg_iou_thresh,
        rpn_batch_size_per_image,
        rpn_positive_fraction,
        rpn_score_thresh,
        # Box parameters
        box_roi_pool,
        box_head,
        box_predictor,
        box_score_thresh,
        box_nms_thresh,
        box_detections_per_img,
        box_fg_iou_thresh,
        box_bg_iou_thresh,
        box_batch_size_per_image,
        box_positive_fraction,
        bbox_reg_weights,
        **kwargs,)

    assert self.model is not None, "Model cannot be initialised as NoneType"

  def override_prediction_heads(self, num_classes: int) -> None:
    # update prediction head so that it's outputs are aligned with number of classes in dataset
    in_features = self.model.roi_heads.box_predictor.cls_score.in_features
    self.model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)

  def load_pretrained_weights(self, weights, weightsClass):
    # the .verify() method is inherited from the WeightsEnum class
    # verify checks that are the weights are not None and are the correct type
    weights = weightsClass.verify(weights)
    # load the weights into the model
    self.model.load_state_dict(weights.get_state_dict(progress=True, check_hash=True))
    # the following function call is required to address
    # the BC-breaking change introduced by the bug-fix at pytorch/vision#2933
    # overwrite_eps is only required when loading pre-trained weights
    overwrite_eps(self.model, 0.0)

  def train(self, num_epochs: int,
                  optimizer: torch.optim,
                  train_dataset: torch.utils.data.DataLoader,
                  val_dataset: Optional[torch.utils.data.DataLoader]= None,
                  is_console_logging: bool = True,
                  calc_running_loss: Optional[bool] = True) -> Dict[str, list]:
    """Trains the model for the specified number of epochs

      Calls function train_batch to train the model on each batch in an epoch.
      Validation loss is calculated at the end of each epoch
      Training time is recorded for the purposes of evaluation.
      Epoch loss is calculates as the average training loss across batches

      Args:
          num_epochs: the number of epochs to train for,

          train_dataset: the PyTorch DataLoader for the training set,

          val_dataset: the PyTorch DataLoader for the validation set, an optional parameter so that
                        this function can be used both in cross validation and hold out contexts

          optimizer : any optimizer imported from torch.optim

          is_console_logging: controls if print statements are sent to the console

          calc_running_loss: a memory management variable - set to False to minimise RAM usage (ie tensor dependencies)

      Returns:
          A dictionary containing 2 lists - one for training loss and another validation loss

      Raises:
          None
      """
    # enforce correct model mode as model will be set to .eval mode to calculate
    # validation loss during training
    self.model.train()
    # track losses across each epoch ready to produce graph
    if calc_running_loss:
      training_losses = []
      val_losses = []

    for epoch in range(num_epochs):
        if is_console_logging:
          # simple progress monitor
          print(f'Epoch {epoch} of {num_epochs}')
        # Keep running total of batch losses ready to take average across epoch
        if calc_running_loss:
          epoch_loss_train = 0

        for batch in train_dataset:
          batch_dict = format_batch(batch, self.model)
          # memory management
          for item in batch:
            del item
          time.sleep(30) #allow time for GPU RAM to be cleared before proceeding

          batch_training_loss = self.__train_batch(batch_dict, optimizer)
          del batch_dict['images'], batch_dict['targets'] # memory management
          time.sleep(30) #allow time for GPU RAM to be cleared before proceeding

          if calc_running_loss:
            epoch_loss_train += batch_training_loss

          del batch_training_loss # memory management
          torch.cuda.empty_cache()
          print('batch complete')

        if calc_running_loss:
          # divide total epoch loss by the total number of batches per epoch
          epoch_training_loss = epoch_loss_train/len(train_dataset)
          del epoch_loss_train
          training_losses.append(epoch_training_loss)


        if is_console_logging and calc_running_loss:
          print(f'Epoch Training loss: {epoch_training_loss}')
        del epoch_training_loss

        if val_dataset is not None:
          val_loss = self.__calc_val_loss(val_dataset_loader)
          val_losses.append(val_loss)

          if is_console_logging:
            print(f'Epoch Validation loss: {val_loss}')
          del val_loss

        # free_gpu_ram ?

    if is_console_logging and calc_running_loss and val_dataset is not None:
      plot_loss([i+1 for i in range(num_epochs)], training_losses, val_losses)

    del training_losses, val_losses
    if val_dataset is not None:
      # this frees up the GPU ready to assign training data
      return {'training_losses': training_losses, 'validation_losses': val_losses}
    else:
      return {'training_losses': training_losses}

  def __train_batch(self, prepped_batch: torch.utils.data.dataset.Subset,
                  optimizer: torch.optim) -> float:
    """Trains the model on a single batch and returns the batch loss

      Calls function format_batch() before training the model on the batch data.
      Total loss is calculated across each of the 4 losses returned by Faster R-CNN
      A backward pass is then completed

      Args:
          train_batch: a subset of the PyTorch dataset

          optimizer : any optimizer imported from torch.optim

      Returns:
          batch loss

      Raises:
          None
      """
    # Calculate training loss
    images, targets = prepped_batch['images'], prepped_batch['targets']

    # use mixed precision to address CUDA out of memory error
    with torch.autocast(device_type="cuda"):
      loss_dict = self.model(images, targets)
      training_loss = sum(v for v in loss_dict.values())
      del loss_dict
    # loss_dict = self.model(images, targets)
    # training_loss = sum(v for v in loss_dict.values())
    del images, targets, prepped_batch

    # backward pass
    optimizer.zero_grad()
    training_loss.backward()

    optimizer.step()

    # detach from computational graph to free up GPU memory
    # use .item() to convert loss to a Python Number ready for plotting
    return training_loss.detach().item()

  ##THE CODE IN FUNCTION EVAL_FORWARD HAS BEEN MODIFIED FROM SOURCE:
  #https://stackoverflow.com/questions/71288513/how-can-i-determine-validation-loss-for-faster-rcnn-pytorch
  #the majority of this function is composed of PyTorch source code that has been slightly modififed so that both losses and predictions are returned during inference
  #PyTorch source files (which part of code was sourced from) can be seen here
  #https://github.com/pytorch/vision/blob/f40c8df02c197d1a9e194210e40dee0e6a6cb1c3/torchvision/models/detection/generalized_rcnn.py#L46

  #analyse using this reference
  #https://stackoverflow.com/questions/60339336/validation-loss-for-pytorch-faster-rcnn

  def __eval_forward(self, images, targets):
      # type: (List[Tensor], Optional[List[Dict[str, Tensor]]]) -> Tuple[Dict[str, Tensor], List[Dict[str, Tensor]]]
      """
      Args:
          images (list[Tensor]): images to be processed
          targets (list[Dict[str, Tensor]]): ground-truth boxes present in the image (optional)
      Returns:
          result (list[BoxList] or dict[Tensor]): the output from the model.
              It returns list[BoxList] contains additional fields
              like `scores`, `labels` and `mask` (for Mask R-CNN models).
      """
      #Added (generalized_rcnn.py forward())
      self.model.eval()

      #*******************************START:Sourced from generalized_rcnn.py forward()
      # lines 72-98 of generalized_rcnn.py
      original_image_sizes: List[Tuple[int, int]] = []
      for img in images:
          val = img.shape[-2:]
          assert len(val) == 2
          original_image_sizes.append((val[0], val[1]))

      images, targets = self.model.transform(images, targets)

      # MODIFICATION: Refactored a block of code into the function check_bboxes
      # Check for degenerate boxes
      check_bboxes(targets)

      features = self.model.backbone(images.tensors)
      if isinstance(features, torch.Tensor):
          features = OrderedDict([("0", features)])
      #*******************************END:Sourced from generalized_rcnn.py forward()

      # MODIFICATION: COMMENTED OUT THESE LINES OF CODE
      # AS RPN AND ROI DO NOT NEED TO BE TRAINING DURING INFERENCE ?
      # model.rpn.training=True
      # model.roi_heads.training=True
      # END OF MODIFICATION


      # ************Following section of code implements similar functionality to:
      # proposals, proposal_losses = model.rpn(images, features, targets)
      # from generalized_rcnn.py forward():
      # Code relating to the RPN (Region Proposal Network)
      #*******************************START:Sourced from rpn.py
      #lines 325-340 in rpn.py
      features_rpn = list(features.values())
      objectness, pred_bbox_deltas = self.model.rpn.head(features_rpn)
      anchors = self.model.rpn.anchor_generator(images, features_rpn)

      num_images = len(anchors)
      num_anchors_per_level_shape_tensors = [o[0].shape for o in objectness]
      num_anchors_per_level = [s[0] * s[1] * s[2] for s in num_anchors_per_level_shape_tensors]
      objectness, pred_bbox_deltas = concat_box_prediction_layers(objectness, pred_bbox_deltas)
      # Following 3 lines are comments from PyTorch source code
      # apply pred_bbox_deltas to anchors to obtain the decoded proposals
      # note that we detach the deltas because Faster R-CNN do not backprop through
      # the proposals
      proposals = self.model.rpn.box_coder.decode(pred_bbox_deltas.detach(), anchors)
      proposals = proposals.view(num_images, -1, 4)
      proposals, scores = self.model.rpn.filter_proposals(proposals, objectness, images.image_sizes, num_anchors_per_level)
      proposal_losses = {}

      #lines 342-351 in rpn.py
      assert targets is not None
      labels, matched_gt_boxes = self.model.rpn.assign_targets_to_anchors(anchors, targets)
      regression_targets = self.model.rpn.box_coder.encode(matched_gt_boxes, anchors)
      loss_objectness, loss_rpn_box_reg = self.model.rpn.compute_loss(
          objectness, pred_bbox_deltas, labels, regression_targets
      )
      proposal_losses = {
          "loss_objectness": loss_objectness,
          "loss_rpn_box_reg": loss_rpn_box_reg,
      }
      #*******************************END:Sourced from rpn.py

      #image size required by ROI head
      image_shapes = images.image_sizes

      # ************Following section of code implements similar functionality to:
      #detections, detector_losses = model.roi_heads(features, proposals, images.image_sizes, targets)
      # from generalized_rcnn.py forward():
      # Code relating to the ROI heads (Region of Interest)
      #*******************************START:Sourced from roi_heads.py
      # lines 745-771 of roi_heads.py
      proposals, matched_idxs, labels, regression_targets = self.model.roi_heads.select_training_samples(proposals, targets)

      box_features = self.model.roi_heads.box_roi_pool(features, proposals, image_shapes)
      box_features = self.model.roi_heads.box_head(box_features)
      class_logits, box_regression = self.model.roi_heads.box_predictor(box_features)

      result: List[Dict[str, torch.Tensor]] = []
      detector_losses = {}

      # The following 2 lines of code were original inside a 'if self.training' condition
      # they have been taken out of this if statment so that
      # losses can be returned when in inference mode (to get val loss)
      loss_classifier, loss_box_reg = fastrcnn_loss(class_logits, box_regression, labels, regression_targets)
      detector_losses = {"loss_classifier": loss_classifier, "loss_box_reg": loss_box_reg}

      # the code below was the corresponding 'else' statement (ie if not self.training)
      # it was taken out if the else statement
      # so that both predictions and losses could be returned when in inference mode
      boxes, scores, labels = self.model.roi_heads.postprocess_detections(class_logits, box_regression, proposals, image_shapes)
      num_images = len(boxes)
      for i in range(num_images):
          result.append(
              {
                  "boxes": boxes[i],
                  "labels": labels[i],
                  "scores": scores[i],
              }
          )
      # TEST HERE
      # conf_scores = []
      # for pred in result:
      #   for index, item in enumerate(pred['labels']):
      #     if item ==2:
      #       conf_scores.append(pred["scores"][index].item())
      # print(conf_scores)

      #*******************************END:Sourced from roi_heads.py

      # rename results as detection so that it is compatible with the subsequent
      # line of code sourced from generalized_rcnn.py
      detections = result
      # Following line of code sourced from generalized_rcnn.py (line 100)
      detections = self.model.transform.postprocess(detections, images.image_sizes, original_image_sizes)  # type: ignore[operator]

      # MODIFICATION: REMOVED THE FOLLOWING 2 LINES OF CODE
      # model.rpn.training=False
      # model.roi_heads.training=False

      #*******************************START:Sourced from generalized_rcnn.py forward()
      # lines 102-104 in generalized_rcnn.py
      losses = {}
      losses.update(detector_losses)
      losses.update(proposal_losses)
      #*******************************END:Sourced from generalized_rcnn.py forward()

      #Added (generalized_rcnn.py forward())
      return losses, detections

  #Function 'evaluate_loss' is modified from source:
  #https://stackoverflow.com/questions/71288513/how-can-i-determine-validation-loss-for-faster-rcnn-pytorch
  # this function has been modified to become a private class method
  # meaining the arg list and some of the internal code has changed
  #A docstring has been added for the modified function
  def __calc_val_loss(self, data_loader: torch.utils.data.DataLoader) -> float:
      """Calculates average validation loss across batches in a PyTorch DataLoader

      Calls function eval_forward() to caluclate validation loss for each batch.
      Each batch validation loss is added to a running total which is divided by the
      number of batches to get average validation loss across batches.
      Calls format_batch to prep input data and move it to the same device as the model (usually GPU)

      Args:
          data_loader: a Pytorch DataLoader

      Returns:
          The average validation loss across batches

      Raises:
          None
      """
      val_loss = 0
      with torch.no_grad():
        for batch in data_loader:
          #*************************************START OF MODIFICATIONS 1
          formatted_batch = format_batch(batch, self.model)
          images, targets = formatted_batch['images'], formatted_batch['targets']
          losses_dict, detections = self.__eval_forward(images, targets)
          #*************************************END OF MODIFICATIONS 1
          losses = sum(loss for loss in losses_dict.values())
          val_loss += losses

      validation_loss = val_loss/ len(data_loader)
      #*************************************START OF MODIFICATIONS 2
      # detach from computational graph to free up GPU memory
      # use .item() to convert loss to a Python Number ready for plotting
      return validation_loss.detach().item()

  def test(self,
           test_dataset_loader: torch.utils.data.DataLoader,
           is_console_logging: bool = True) -> dict:
    """Calculates model mAP at different IoU thresholds on the given test set

      Sets the model to eval mode and runs the model to get inferences.
      Non-maximum suppression is applied before mAPs are calculated.
      mAP50 and mAP50-95 are printed for convenience
      Model predictions are returned so that they can be visualised by
      a follow up call to visualise_predictions()

      Args:
          test_dataset_loader: the Pytorch DataLoader for the test set
          is_console_logging: controls if print statements are sent to the console

      Returns:
          Dictionary with 3 keys ; model predictions, model evaluation metrics (mAP) and image paths
          (to support visualisations of predictions on original imgs)

      Raises:
          None
      """

    # according to the PyTorch docs:
    # "be sure to call model.eval() method before inferencing to set the dropout and batch normalization layers to evaluation mode.
    # Failing to do this will yield inconsistent inference results."
    # https://pytorch.org/tutorials/beginner/basics/saveloadrun_tutorial.html#:~:text=Saving%20and%20Loading%20Model%20Weights&text=Using%20weights_only%3DTrue%20is%20considered%20a%20best%20practice%20when%20loading%20weights.&text=be%20sure%20to%20call%20model,will%20yield%20inconsistent%20inference%20results.
    self.model.eval()

    # memory management
    torch.cuda.empty_cache()

    inferences_times = []
    all_preds = []
    all_targets = []
    test_imgs = []
    metric_map50 = MeanAveragePrecision(box_format="xyxy",iou_type="bbox", iou_thresholds=[0.5], class_metrics=True, extended_summary=True)
    #no iou_threshold leads to 50-95 coco mAP
    metric_map50_95 = MeanAveragePrecision(box_format="xyxy",iou_type="bbox", class_metrics=True, extended_summary=True)

    #reduce memory usage by not storing intermediate tensors needed to calculate gradients
    #torch.no_grad reportedly speeds up computations
    with torch.no_grad():
      for batch in test_dataset_loader:
          # gather all images into a single array
          formatted_data = format_batch(batch, self.model)
          batch_images, batch_targets = formatted_data['images'], formatted_data['targets']

          start_time = time.time()
          predicted = self.model(batch_images)
          # calculate average inference time for an image
          end_time = time.time()
          duration = execution_time = end_time - start_time
          test_size = len(test_dataset_loader)*test_dataset_loader.batch_size
          avg_time = duration / test_size
          inferences_times.append(avg_time)
          all_preds.extend(predicted)
          all_targets.extend(batch_targets)
          test_imgs.extend(batch_images)

      # Update metric with predictions and respective ground truth
      metric_map50.update(all_preds, all_targets)
      metric_map50_95.update(all_preds, all_targets)
      # Compute the results
      map50_result = metric_map50.compute()
      map_50_95_result = metric_map50_95.compute()

      if is_console_logging:
        print('mAP50:')
        print(map50_result['map'])

        print('Class wise metrics: mAP50:')
        print(map50_result['map_per_class'])
        print("\n")
        print('mAP 50-95:')
        print(map_50_95_result['map'])

        print('Class wise metrics: mAP50-95:')
        print(map_50_95_result['map_per_class'])

        print('Average Inference time:')
        print(sum(inferences_times)/len(inferences_times))

      # fig_, ax_ = metric.plot()
      return {"predictions": predicted, "metrics": {"mAP50": map50_result, "mAP50-95": map_50_95_result}, "test_imgs": test_imgs}

### Cross validation code:

In [23]:
from sklearn.model_selection import KFold

# add wandB logging here ?
def k_fold_training(train_dataset: UAVImageDataset,
                    n_splits: int,
                    random_state: int,
                    batch_size: int,
                    # model parameters
                    backbone: nn.Module,
                    anchor_generator: torchvision.models.detection.rpn.AnchorGenerator,
                    roi_pooler: torchvision.ops.MultiScaleRoIAlign,
                    num_epochs: 15) -> dict[str, float]:
    """Performs K-fold cross validation and returns a dictionary of average mAP scores across folds

      A new Faster R-CNN model is instantiated for each fold - based on the input parameters

      Args:
          train_dataset: instance of the UAVImageDataset class,
          n_splits: number of splits for k-fold cross validation
          random_state: ensures splits are reproducible
          batch_size: batch size for both training and testing
          backbone: a custom neural network backbone for the Faster R-CNN model
          anchor_generator: the custom anchor generator to go with the custom backbone
          roi_pooler: the custom roi_pooler to go with the custom backbone,
          num_epochs: number of epochs to train the model for

      Returns:
          Dictionary with 6 keys ; metrics for average, plastic and water hyacinth - at two IoU
          thresholds mAP50 and mAP50-95

      Raises:
          None
      """
    mAP50_scores = []
    plastic_map50_scores = []
    water_hyacinth_map50_scores = []

    mAP50_95_scores = []
    plastic_map50_95_scores = []
    water_hyacinth_map50_95_scores = []

    kf = KFold(n_splits=n_splits, shuffle=True, random_state=random_state)

    for fold, (train_idx, test_idx) in enumerate(kf.split(train_dataset)):
      print(f"Fold {fold + 1} of {n_splits}")

      # Define the data loaders for the current fold
      subsampled_train_dataset = torch.utils.data.Subset(train_dataset, train_idx)
      train_dataset_loader = torch.utils.data.DataLoader(subsampled_train_dataset,
                                                   collate_fn=lambda x: x,
                                                   batch_size=batch_size,
                                                   shuffle=True,
                                                   pin_memory=True,
                                                  num_workers=12,
                                                   prefetch_factor=10
                                                   )

      subsampled_test_dataset = torch.utils.data.Subset(train_dataset, test_idx)
      test_dataset_loader = torch.utils.data.DataLoader(subsampled_test_dataset,
                                                   collate_fn=lambda x: x,
                                                   batch_size=batch_size,
                                                   shuffle=True,
                                                   pin_memory=True,
                                                  num_workers=12,
                                                   prefetch_factor=10
                                                   )

      # initialise a model based on the parameters and move to device
      model_wrapper = FasterRCNNWrapper(
          backbone=backbone,
          num_classes=3,
          rpn_anchor_generator=anchor_generator,
          box_roi_pool=roi_pooler
      )
      num_classes=3 #2 classes +1 to account for background
      model_wrapper.override_prediction_heads(num_classes)
      model_wrapper.model.to(device)

      # initialise optimizer
      optimizer = torch.optim.Adam(model_wrapper.model.parameters(), lr=0.00025)
      num_epochs = 15

      # train
      model_wrapper.train(num_epochs=num_epochs,
                          train_dataset=train_dataset_loader,
                          optimizer=optimizer,
                          is_console_logging=False)

      # test
      model_wrapper.model.eval()
      test_results = model_wrapper.test(test_dataset_loader=test_dataset_loader,
                         is_console_logging=False)

      # push mAP to test_scores ready to calculate average acrosss folds
      # average results across classes
      mAP50_scores.append(test_results['metrics']['mAP50']['map'])
      mAP50_95_scores.append(test_results['metrics']['mAP50-95']['map'])

      # class wise results
      plastic_map50_scores.append(test_results['metrics']['mAP50']['map_per_class'][0])
      plastic_map50_95_scores.append(test_results['metrics']['mAP50-95']['map_per_class'][0])
      water_hyacinth_map50_scores.append(test_results['metrics']['mAP50']['map_per_class'][1])
      water_hyacinth_map50_95_scores.append(test_results['metrics']['mAP50-95']['map_per_class'][1])

      free_gpu_ram(train_dataset_loader)
      free_gpu_ram(test_dataset_loader)

    # calculate average test scores across folds
    return {'avgMAP50': sum(mAP50_scores) / len(mAP50_scores),
            'avgMAP50-95': sum(mAP50_95_scores) / len(mAP50_95_scores),
            'avgPlastic_mAP50': sum(plastic_map50_scores) / len(plastic_map50_scores),
            'avgWaterHyacinth_mAP50': sum(water_hyacinth_map50_scores) / len(water_hyacinth_map50_scores),
            'avgPlastic_mAP50-95': sum(plastic_map50_95_scores) / len(plastic_map50_95_scores),
            'avgWaterHyacinth_mAP50-95': sum(water_hyacinth_map50_95_scores) / len(water_hyacinth_map50_95_scores)}

## Function to Test the robustness of the model

In [24]:
import matplotlib.pyplot as plt



def model_robustness_curve(anchor_generator: torchvision.models.detection.rpn.AnchorGenerator,
                           roi_pooler: torchvision.ops.MultiScaleRoIAlign,
                           backbone: torch.nn.Module,
                          train_dataset: UAVImageDataset,
                           list_dataset_subsamples: List[float],
                           num_epochs: int,
                           val_dataset_loader: torch.utils.data.DataLoader,
                           test_dataset_loader: torch.utils.data.DataLoader,) -> Dict[str, float]:
    """Analyses model performance across different sizes of training datasets

      Trains the model on the specified subsets and creates a 'robsutness curve' showing mAP50-95 of each model
      The model is re-initialised for each train/test on dataset subsample ; so that the test set effectively
      remains unseen by the model

      Args:
          anchor_generator: the anchor generator required for model initialisation
          roi_pooler: the ROI pooler required for model initialisation
          backbone: the custom backbone required for model initialisation
          train_dataset: an instance of the UAVImageDataset which has been loaded with data
                          required to be torch.utils.Dataset type so that torch.utils.data.Subset can be used to subsample,
            list_dataset_subsamples: the sizes of each subsample represented as decimals - ie 0.1 == 10%,
            num_epochs: number of epochs to train the model for,
            val_dataset_loader: validation dataset loader,
            test_dataset_loader: test dataset loader

      Returns:
          Dictionary

      Raises:
          None
      """

    training_set_sizes = []
    results = []
    for decimal in list_dataset_subsamples:
      size_training_set = round(len(train_dataset)*decimal)
      training_set_sizes.append(size_training_set)
      indices_to_extract = [i for i in range(size_training_set)]
      subsampled_dataset = torch.utils.data.Subset(train_dataset, indices_to_extract)
      subset_train_dataset_loader = torch.utils.data.DataLoader(subsampled_dataset,
                                                   collate_fn=lambda x: x,
                                                   batch_size=16,
                                                   shuffle=True,
                                                   pin_memory=True,
                                                  num_workers=12,
                                                   prefetch_factor=10
                                                   )

      # re-initialise the model each time so that the test dataset remains unseen by the model
      model_wrapper = FasterRCNNWrapper(
          backbone=backbone,
          num_classes=3,
          rpn_anchor_generator=anchor_generator,
          box_roi_pool=roi_pooler
      )
      num_classes=3 #3 classes +1 to account for background
      model_wrapper.override_prediction_heads(num_classes)

      model_wrapper.model.to(device)
      optimizer = torch.optim.SGD(model_wrapper.model.parameters() , lr=0.0005, momentum=0.9, weight_decay=0.0005)
      test_results = model_pipeline(model_wrapper, optimizer, num_epochs, subset_train_dataset_loader,  val_dataset_loader, test_dataset_loader, device, False, "Faster R-CNN Tracker", is_console_logging=False)
      results.append(test_results['metrics']['mAP50-95']['map'].item())

    # draw graph
    plt.plot(training_set_sizes, results)
    plt.title("Robustness Curve: Different sizes of dataset")
    plt.xlabel("Number of images in training dataset")
    plt.ylabel("mAP50-95 (average across the 2 classes)")
    plt.show()

## Definition of Model Pipeline: <a class="anchor" name="model_pipeline"></a>

In [25]:
!pip install wandb -qU
!pip install onnx

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.0/16.0 MB 46.5 MB/s eta 0:00:00


In [26]:
import wandb
wandb.login()

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: otterhian (otterhian-student) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


True

In [27]:
from torch.utils.tensorboard import SummaryWriter # to print to tensorboard
import onnx

def model_pipeline(model_wrapper: FasterRCNNWrapper,
                   optimizer: torch.optim,
                   num_epochs: int,

                   train_dataset_loader,
                   val_dataset_loader,
                   test_dataset_loader,

                   device: torch.device,
                   is_logging: bool,
                   project_name: str,
                   is_console_logging: bool = True) -> Dict:

  # Set up TensorBoard and Wandb logging
  if is_logging:
    log_dir = "/content/drive/MyDrive/Colab Notebooks/UoL FP/my_data_model_test/tensorboard_runs/FasterRCNN/"

    # Prep identifiers for the run
    train_size = len(train_dataset_loader) * train_dataset_loader.batch_size
    val_size = len(val_dataset_loader) * val_dataset_loader.batch_size
    test_size = len(test_dataset_loader) * test_dataset_loader.batch_size
    time_stamp = datetime.datetime.now().strftime("%m_%d-%H_%M")

    writer = SummaryWriter(f"{log_dir}/{train_size}_{val_size}_{test_size}____{time_stamp}")

    # gather hyperparameters used in this run
    optimizer_hyperparams = optimizer.state_dict()['param_groups'][0] #dict is first item in array

    model = model_wrapper.model
    model_anchor_generator = model.rpn.anchor_generator
    train_set_size = train_dataset_loader.batch_size*len(train_dataset_loader)
    val_set_size = val_dataset_loader.batch_size*len(val_dataset_loader)
    test_set_size = test_dataset_loader.batch_size*len(test_dataset_loader)

    if optimizer_hyperparams.get('momentum') == None: #ADAM does not have property momentum
      learning_rate, weight_decay = optimizer_hyperparams['lr'], optimizer_hyperparams['weight_decay']
      hyperparameters = {
          "num_epochs": num_epochs,
          "train_val_test_split": f"{train_set_size}/{val_set_size}/{test_set_size}",
          "optimizer_lr": learning_rate,
          "optimizer_weight_decay": weight_decay,
          # TensorBoard only accepts datatypes int, float, str, bool, or torch.Tensor
          # for the method add_hparams()
          # convert the following tuples and lists to strings
          "anchor_aspect_ratios": str(model_anchor_generator.aspect_ratios),
          "num_anchors_per_loc": str(model_anchor_generator.num_anchors_per_location()),
          "anchor_sizes": str(model_anchor_generator.sizes)

      }
    else: #SGD does have property momentum
      learning_rate, momentum, weight_decay = optimizer_hyperparams['lr'], optimizer_hyperparams['momentum'], optimizer_hyperparams['weight_decay']
      hyperparameters = {
          "num_epochs": num_epochs,
          "train_val_test_split": f"{train_set_size}/{val_set_size}/{test_set_size}",
          "optimizer_lr": learning_rate,
          "optimizer_momentum": momentum,
          "optimizer_weight_decay": weight_decay,
          # TensorBoard only accepts datatypes int, float, str, bool, or torch.Tensor
          # for the method add_hparams()
          # convert the following tuples and lists to strings
          "anchor_aspect_ratios": str(model_anchor_generator.aspect_ratios),
          "num_anchors_per_loc": str(model_anchor_generator.num_anchors_per_location()),
          "anchor_sizes": str(model_anchor_generator.sizes)

      }

    # initialise wandb before training so that it monitors CPU usage
    # use 'with wandb.init' context manager so that wandb.finish() is automatically
    # called at the end of the block
    with wandb.init(project=project_name, config=hyperparameters):
      # track gradients and parameters
      # wandb.watch(modelWrapper.model, log="all") #NOT WORKING WITH CUSTOM BACKBONE

      # train the model and measure training time to log
      start_time = time.time()
      losses_dict = model_wrapper.train(num_epochs, optimizer, train_dataset_loader, val_dataset_loader, is_console_logging)
      end_time = time.time()
      execution_time = end_time - start_time

      # delete training data from GPU RAM ready for testing
      free_gpu_ram(train_dataset_loader)
      free_gpu_ram(val_dataset_loader)

      # log training time
      wandb.log({"Training time": execution_time})
      writer.add_scalar("Training time", execution_time)

      training_losses, val_losses = losses_dict['training_losses'], losses_dict['validation_losses']
      # log training data
      for i in range(len(training_losses)):
            # tensorboard log
            writer.add_scalar('Training Loss', training_losses[i], i)
            writer.add_scalar('Validation Loss', val_losses[i], i)
            writer.add_scalars( 'Training vs. Validation Loss',
                                {'Training' : training_losses[i], 'Validation' : val_losses[i] },
                                i)
      # Wandb log
      # log train vs val loss separately so they they can be visually compared with different runs
      wandb.log({"train_loss" : wandb.plot.line_series(
                  xs=list(range(num_epochs)),
                  ys=[training_losses],
                  title="Training loss",
                  xname="Epochs")})

      wandb.log({"val_loss" : wandb.plot.line_series(
                  xs=list(range(num_epochs)),
                  ys=[val_losses],
                  title="Validation loss",
                  xname="Epochs")})

      # plot together on same graph for direct comparison of train and val loss for a run
      wandb.log({"train_vs_val_loss" : wandb.plot.line_series(
                  xs=list(range(num_epochs)),
                  ys=[training_losses, val_losses],
                  keys=["Training", "Validation"],
                  title="Train vs val loss",
                  xname="Epochs")})

      # test the model and measure execution time so avg inference time can be calculated
      start_time = time.time()
      test_results = model_wrapper.test(test_dataset_loader, is_console_logging)
      end_time = time.time()
      execution_time = end_time - start_time
      # log testing data

      # Save the model in the exchangeable ONNX format
      free_gpu_ram(test_dataset_loader)
      # GPU RAM not capable of tracing pre-trained model through input
      # test_images = [item[0].to(device=next(model.parameters()).device) for batch in test_dataset_loader for item in batch]
      # torch.onnx.export(modelWrapper.model, test_images, "model.onnx") #test_results["test_imgs"]
      # wandb.save("model.onnx")

      # memory management
      # for img in test_images:
      #   del img
      # torch.cuda.empty_cache()
      # writer.add_graph(modelWrapper.model, test_images)

      # calculate average inference time and log
      wandb.log({"Inference time": execution_time/test_set_size})
      writer.add_scalar("Inference", execution_time/test_set_size)

      # TensorBoard Log
      # UPDATE HERE ********************** FOR NEW RETURN STATEMENT FROM TEST FUNC
      # info
      # print('mAP50:')
      # print(map50_result['map'])

      # print('Class wise metrics: mAP50:')
      # print(map50_result['map_per_class'])

      # print('mAP 50-95:')
      # print(map_50_95_result['map'])

      # print('Class wise metrics: mAP50-95:')
      # print(map_50_95_result['map_per_class'])

      # # fig_, ax_ = metric.plot()
      # return {"predictions": predicted, "metrics": {"mAP50": map50_result, "mAP50-95": map_50_95_result},



      avg_map50 = test_results['metrics']['mAP50']['map']
      plastic_mAP50 = test_results['metrics']['mAP50']['map_per_class'][0]
      water_hyacinth_mAP50 = test_results['metrics']['mAP50']['map_per_class'][1]

      avg_mAP50_95 = test_results['metrics']['mAP50-95']['map']
      plastic_mAP50_95 = test_results['metrics']['mAP50-95']['map_per_class'][0]
      water_hyacinth_mAP50_95 = test_results['metrics']['mAP50-95']['map_per_class'][1]
      test_metrics = {"avg mAP50": avg_map50,
                 "plastic mAP50": plastic_mAP50,
                 "water hyacinth mAP50": water_hyacinth_mAP50,

                 "avg mAP50-95": avg_mAP50_95,
                 "plastic mAP50-95": plastic_mAP50_95,
                 "water hyacinth mAP50-95": water_hyacinth_mAP50_95
                 }

      writer.add_hparams(hyperparameters, test_metrics)

      # Wandb log
      wandb.log(test_metrics)
      # log code files in case extra info is needed for analysis
      wandb.run.log_code("/content/drive/MyDrive/Colab Notebooks/UoL FP/my_data_model_test", include_fn=lambda path: path.endswith(".ipynb"))

        # Write image data to TensorBoard log dir
        # writer.add_image( 'Four Fashion-MNIST Images', img_grid)

      # free up memory as writer no longer needed
      writer.close()
      if not is_console_logging:
        return test_results

  else:
    model_wrapper.train(num_epochs, optimizer, train_dataset_loader, val_dataset_loader, is_console_logging)
    # memory management
    # clear training data off GPU reading for testing data
    free_gpu_ram(train_dataset_loader)

    test_results = model_wrapper.test(test_dataset_loader, is_console_logging)
    free_gpu_ram(test_dataset_loader)

    if not is_console_logging:
      return test_results

  # visualise samples of predictions
  # model_predictions = test_results["predictions"]
  # for i in range(len(model_predictions)):
  #   visualisePredictions(i, test_dataset, model_predictions)

In [28]:
stop

NameError: name 'stop' is not defined

## Load Data:

In [29]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [30]:
# set deterministic behaviour
set_seeds()

In [ ]:
# faster loading times
train_dataset = UAVImageDataset(data_dir_imgs="/content/drive/MyDrive/UoL Final Project/Project Data/official project data/2 classes - image size test/train/images",
                                df_path="/content/drive/MyDrive/UoL Final Project/Project Data/official project data/2 classes - image size test/train_pd.csv",
                                classes_start_at_zero=False)

val_dataset = UAVImageDataset(data_dir_imgs="/content/drive/MyDrive/UoL Final Project/Project Data/official project data/2 classes - image size test/valid/images",
                                df_path="/content/drive/MyDrive/UoL Final Project/Project Data/official project data/2 classes - image size test/valid_pd.csv",
                              classes_start_at_zero=False)

test_dataset = UAVImageDataset(data_dir_imgs="/content/drive/MyDrive/UoL Final Project/Project Data/official project data/2 classes - image size test/test/images",
                                df_path="/content/drive/MyDrive/UoL Final Project/Project Data/official project data/2 classes - image size test/test_pd.csv",
                               classes_start_at_zero=False)

In [ ]:
train_dataset_loader = torch.utils.data.DataLoader(train_dataset,
                                                   collate_fn=lambda x: x,
                                                   batch_size=8,
                                                   shuffle=True,
                                                   pin_memory=True,
                                                  num_workers=12,
                                                   prefetch_factor=10
                                                   )


val_dataset_loader = torch.utils.data.DataLoader(val_dataset, collate_fn=lambda x: x, batch_size=8,
                                                  num_workers=6)

test_dataset_loader = torch.utils.data.DataLoader(test_dataset, collate_fn=lambda x: x, batch_size=8,
                                                  num_workers=6)

## Establishing a meaningful baseline of performance: <a class="anchor" name="fasterRCNN_baseline"></a>

### Pre-trained Faster R-CNN baseline performance:

In [ ]:
from torchvision.models import resnet50
from torchvision.models.detection import FasterRCNN
from torchvision.models._utils import _ovewrite_value_param
from torchvision.models.detection.rpn import AnchorGenerator
from torchvision.models.detection.faster_rcnn import FasterRCNN_ResNet50_FPN_Weights, FastRCNNPredictor
from torchvision.models.detection.backbone_utils import _resnet_fpn_extractor, _validate_trainable_layers
from torchvision.models.detection._utils import overwrite_eps
from torchvision.ops import misc as misc_nn_ops
import torch.nn as nn

num_trainable_backbone_layers = 3 # valid range for resnet50 is 0-5 according to PyTorch docs
backbone = resnet50(weights=None, progress=True, norm_layer=misc_nn_ops.FrozenBatchNorm2d)
backbone = _resnet_fpn_extractor(backbone, num_trainable_backbone_layers)

#use default values for a baseline implementation
anchor_sizes = ((32,), (64,), (128,), (256,), (512,))

# anchor_sizes = ((16,), (32,), (64,), (128,), (512,))

# default values
anchor_aspect_ratios = ((0.5, 1.0, 2.0),) * len (anchor_sizes)

anchor_generator = AnchorGenerator(
    sizes=anchor_sizes,
    aspect_ratios=anchor_aspect_ratios
)
#use default values for a baseline implementation
roi_pooler = torchvision.ops.MultiScaleRoIAlign(featmap_names=["0", "1", "2", "3"],
                                                output_size=7,
                                                sampling_ratio=2)

from torchvision.models.detection.roi_heads import RoIHeads
custom_roi_heads = RoIHeads(
            box_roi_pool = None,
            box_head = None,
            box_predictor = None,
            # Faster R-CNN training
            fg_iou_thresh=0.5,
            bg_iou_thresh=0.5,
            batch_size_per_image=512,
            positive_fraction=0.25,
            bbox_reg_weights=None,
            # Faster R-CNN inference
            score_thresh=0.001, #default is 0.05,
            nms_thresh=0,  #default is 0.5,
            detections_per_img=100
        )

# # Box parameters
#         box_roi_pool=None,
#         box_head=None,
#         box_predictor=None,
#         box_score_thresh=0.05,
#         box_nms_thresh=0.5,
#         box_detections_per_img=100,
#         box_fg_iou_thresh=0.5,
#         box_bg_iou_thresh=0.5,
#         box_batch_size_per_image=512,
#         box_positive_fraction=0.25,
#         bbox_reg_weights=None,




custom_model_wrapper = FasterRCNNWrapper(
    backbone=backbone,
    num_classes=91,
    rpn_anchor_generator=anchor_generator,
    box_roi_pool=roi_pooler,
    # TESTING
    # box_predictor=FastRCNNPredictor(1024, 3) #seems like this could replace the override_prediction_heads method?
    # roi_heads=custom_roi_heads


    # box_score_thresh=0.001,
    # box_nms_thresh=0,

)
# use default weights trained on COCO image dataset
weights = FasterRCNN_ResNet50_FPN_Weights.DEFAULT
custom_model_wrapper.load_pretrained_weights(weights, FasterRCNN_ResNet50_FPN_Weights)
#override the prediction heads to have the correct number of classes
num_classes=3 #3 classes +1 to account for background
custom_model_wrapper.override_prediction_heads(num_classes)

# print(custom_model_wrapper.model.roi_heads.nms_thresh)
# print(custom_model_wrapper.model.roi_heads.score_thresh)

In [ ]:
# #Must move model (and it's parameters) to the GPU before initialisating the optimizer
# #according to a video on the official PyTorch Youtube Channel
# #https://www.youtube.com/watch?v=jF43_wj_DCQ&t=213s
# #otherwise the optimiser will update the wrong set of model parameters (on the CPU)
custom_model_wrapper.model.to(device)
# # try this for optimizer
# # filter(lambda p: p.requires_grad, model.parameters())
# # https://medium.com/we-talk-data/guide-to-freezing-layers-in-pytorch-best-practices-and-practical-examples-8e644e7a9598#:~:text=To%20avoid%20this%2C%20verify%20the%20requires_grad%20status%20of%20each%20layer.&text=By%20running%20this%20code%2C%20you,ve%20frozen%20the%20correct%20ones.&text=If%20you%20don't%20filter,parameters%20that%20won't%20change.
optimizer = torch.optim.SGD(custom_model_wrapper.model.parameters() , lr=0.0005, momentum=0.9, weight_decay=0.0005)
num_epochs = 15
model_pipeline(custom_model_wrapper, optimizer, num_epochs, train_dataset_loader,  val_dataset_loader, test_dataset_loader, device, False, "Faster R-CNN Tracker")

In [ ]:
resnet_50_path = '/content/drive/MyDrive/Colab Notebooks/UoL FP/my_data_model_test/saved_models/resnet_50_weights.pth'
torch.save(custom_model_wrapper.model.state_dict(), resnet_50_path)

In [ ]:
#code sourced from
#https://saturncloud.io/blog/check-the-total-number-of-parameters-in-a-pytorch-model/#:~:text=To%20check%20the%20number%20of%20parameters%20in%20a%20PyTorch%20model,its%20total%20number%20of%20elements.
total_params = sum(p.numel() for p in custom_model_wrapper.model.parameters())
print(f"Number of parameters: {total_params}")

### Pre-trained YOLO baseline performance:

In [ ]:
!pip install ultralytics --no-deps

In [ ]:
!pip install thop --no-deps

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# use the default parameters specified by docs: https://docs.ultralytics.com/usage/cfg/#train-settings
# this gives an idea of how YOLOv12 would perform 'out of the box'
hyperparameters = {
      "epochs":15,
      # Optimizer Options include SGD, Adam, AdamW, NAdam, RAdam, RMSProp etc., or auto for automatic selection based on model configuration.
      "optimizer": "auto",
      "img_size": 640, #preferred images size according to github
      "batch_size":16,
      "weight_decay":0.0005,
      "warm_up_epochs" : 3.0,
  }

In [ ]:
from ultralytics import YOLO
import time

with wandb.init(project="Updated YOLO Tracker", config=hyperparameters):
    # Load a model
    model = YOLO("yolo12s.pt") #use the latest version of YOLO

    # Train the model
    start_time = time.time()

    train_results = model.train(
        data="/content/drive/MyDrive/UoL Final Project/Project Data/official project data/UAV riverine plastic 2 - null images removed - resized-to-2048x1535/data.yaml",
        epochs = hyperparameters["epochs"], # number of training epochs
        imgsz = hyperparameters["img_size"], # training image size
        batch = hyperparameters["batch_size"],
        weight_decay = hyperparameters["weight_decay"],
        warmup_epochs = hyperparameters["warm_up_epochs"],
        plots=True, #save plots for analysis
        device= "cuda:0"
        )

    end_time = time.time()
    execution_time = end_time - start_time
    # log training time
    wandb.log({"Training time": execution_time})

    class_wise_metrics_map50 = {}
    class_wise_metrics_map95 = {}
    for class_index in train_results.ap_class_index:
      map50, map50_95 = train_results.class_result(class_index)[2], train_results.class_result(class_index)[3]
      class_wise_metrics_map50[f"{train_results.names[class_index]}_map50"] = map50
      class_wise_metrics_map95[f"{train_results.names[class_index]}_map95"] = map50_95


    wandb.log(class_wise_metrics_map50)
    wandb.log(class_wise_metrics_map95)
    wandb.log(train_results.results_dict)

    wandb.log({"PR_curve": wandb.Image(f"/content/runs/detect/train/PR_curve.png")})
    wandb.log({"Results": wandb.Image(f"/content/runs/detect/train/results.png")})

## Custom Backbone 1: Residual connections + CBAM + FPN:

### Definition of Residual Blocks:

In [31]:
from torchvision.models import resnet50, ResNet50_Weights
from torchvision.models.detection import FasterRCNN
from torchvision.models._utils import _ovewrite_value_param
from torchvision.models.detection.rpn import AnchorGenerator
from torchvision.models.detection.faster_rcnn import FasterRCNN_ResNet50_FPN_Weights, FastRCNNPredictor
from torchvision.models.detection.backbone_utils import _resnet_fpn_extractor, _validate_trainable_layers
from torchvision.models.detection._utils import overwrite_eps
from torchvision.ops import misc as misc_nn_ops
import torch.nn as nn
from torch.nn import functional as F
from torch.nn import Conv2d, BatchNorm2d, LeakyReLU, ReLU, LazyConv2d

#  output size of conv layer = [(input size + 2(padding) - kernel size)/stride]  +1

# the ResNet block proposed in the original paper https://arxiv.org/pdf/1512.03385
# 3x3 convolutional layers with stride of 1 and padding of 1
# followed by batch normalisation and activation function
# then 3x3 convolutional layer
# batch normalisation
# residual connection (add block inputs to current set of features)
# final activation function

# CODE IN THE FOLLOWING CELLS IS INFORMED BY
# https://debuggercafe.com/traffic-sign-detection-using-pytorch-faster-rcnn-with-custom-backbone/
# https://debuggercafe.com/traffic-sign-recognition-using-custom-image-classification-model-in-pytorch/

# try ReLU or ELU ?
    # ReLU not great for intermediate layers in a network as
    # it suffers from 'dying ReLUs'
    # https://www.analyticsvidhya.com/blog/2021/06/the-challenge-of-vanishing-exploding-gradients-in-deep-neural-networks/
    # x = F.relu(self.bn1(self.conv1(x)))

class IdentityBlock(nn.Module):
  """
  An identity conv block with a residual connection ;
  based on the original paper ResNet https://arxiv.org/pdf/1512.03385

  This block processes an image without changing dimensions;
  The block has 2 convolutional layers + batch normalisation
  Kernel size is consistent across the 2 convolutional layers
  (kernel size must remain the same otherwise input dimensions would not match output dimensions)
  stride is 1 for each conv layer - otherwise iput shape will change and raw input dimension will
  not be compatible with latest layer output. This means residual connection will fail.

  Attributes:
  - 2 conv layers
  - 2 batch normalisation layers
  """
  def __init__(self,
               num_filters : int,
               kernel : int,
               input_padding : int
               ) -> None:

    """Initalises layers required for forward pass

    Args:
      num_filters: the number of filters (out_channels) for every conv layer
      kernel: kernel size for each conv layer
      input_padding: padding for every conv layer

    Returns: None
    """
    super().__init__()

    #formula for output shape of conv2d:
    # o = [i + 2*p - k - (k-1)*(d-1)]/s + 1
    # i= input dim, p=padding, k=kernel, d=dilation , s=stride
    # source: https://discuss.pytorch.org/t/how-to-keep-the-shape-of-input-and-output-same-when-dilation-conv/14338
    # in order for input dimensions to remain the same:
    # if stride is 1:  o = [i + 2*p - k - (k-1)*(d-1)] + 1
    # for input.dim to equal output.dim, then 2*p - k - (k-1)*(d-1) = -1 ; so that o =  i-1+1
    # dilation is 1 by default: 2*p - k = -1
    # so 2p = k-1  in order for input shape not to change ?
    assert 2*input_padding == kernel-1, "Padding and kernel size not suitable for identity block, these p and k values will change input dimensions"

    self.conv1 = LazyConv2d(
                        out_channels = num_filters,
                        kernel_size = kernel,
                        stride = 1,
                        padding = input_padding
                        )

    self.conv2 = LazyConv2d(
                        out_channels = num_filters,
                        kernel_size = kernel,
                        stride = 1,
                        padding = input_padding
                        )
    # batch normalisation has learnable parameters ('affine parameters') so create
    # separate batch normalisation layers for each conv layer
    self.bn1 = BatchNorm2d(num_filters)
    self.bn2 = BatchNorm2d(num_filters)

  def forward(self, x: torch.tensor) -> torch.tensor:
    # save original input ready to add at residual connection
    identity = x

    # LAYER 1
    x = self.conv1(x)
    x = self.bn1(x)
    x = F.relu(x)

    # LAYER 2
    x = self.conv2(x)
    x = self.bn2(x)

    x += identity #residual connection

    x = F.relu(x)
    return x

In [32]:
class ConvBlock(nn.Module):
  """
  A conv block with a residual connection;
  based on the original paper ResNet https://arxiv.org/pdf/1512.03385

  The block has 2 convolutional layers + batch normalisation
  The third conv layer is to resize the original input so it can be added as a residual connection

  Attributes:
  - 3 conv layers
  - 2 batch normalisation layers
  """
  def __init__(self,
               num_filters : int,
               kernel : int,
               stride_val : int ,
               input_padding : int
               ) -> None:
    """
    Initialises the layers require by forward()

    Args:
    num_filters: the number of filters for every conv layer
    kernel: kernel size for every conv layer
    stride_val: kernel size for the first and last conv layer
    input_padding: padding size for the first 2 conv layers

    Returns: None
    """
    super().__init__()
    self.conv1 = LazyConv2d(
                        out_channels = num_filters,
                        kernel_size = kernel,
                        stride = stride_val,
                        padding = input_padding
                        )

    self.conv2 = LazyConv2d(out_channels = num_filters,
                            kernel_size = kernel,
                            #use defualt stride value,
                            padding = input_padding
                            )

    # use a 1x1 conv layer for 'cross channel downsampling'
    # the 1x1 conv will be used to effectively reshape the inputs ('identity') so that
    # it can be added to the output of the layer preceding the residual connection
    self.conv_input_transformer = LazyConv2d(
                                        out_channels = num_filters, #match output shape of last layer,
                                        kernel_size = 1,
                                        stride = stride_val,
                                        padding = 0
                                        )
    # batch normalisation has learnable parameters ('affine parameters') so create
    # separate batch normalisation layers for each conv layer
    self.bn1 = BatchNorm2d(num_filters)
    self.bn2 = BatchNorm2d(num_filters)


  def forward(self, x: torch.tensor) -> torch.tensor:
    # save original input ready to add at residual connection
    identity = x

    # LAYER 1
    x = self.conv1(x)
    x = self.bn1(x)
    x = F.relu(x)

    # LAYER 2
    x = self.conv2(x)
    x = self.bn2(x)

    x += self.conv_input_transformer(identity) #residual connection
    x = F.relu(x)

    return x


### Definition of Spatial and Channel Attention:

In [33]:
#SAM
# 1. channel pool: input tensor (cxhxw) is decomposed to (2xhxw)
# the two channels represent max pooling and average pooling across all channels

# 2. conv layer which outputs a 1 channel feature map (1xhxw)
# 3. batch normalisation layer to normalise and scale output of conv layer

# 4. option to use relu activation function after conv layer

# 5. output is passed to sigmoid activation fn

# 6. element-wise multiplication of spatial attention mask and all feature maps in input tensor

# https://github.com/Jongchan/attention-module/blob/master/MODELS/cbam.py
class SpatialAttention(nn.Module):
  def __init__(self, kernel_size:int = 7,
               conv_bias: bool = False,
               has_batch_norm: bool = True,
               has_relu: bool = True) -> None:
    super().__init__()

    self.conv = nn.Conv2d(2, 1, kernel_size=kernel_size, stride=1, padding=(kernel_size-1) // 2, bias=conv_bias)
    self.bn = nn.BatchNorm2d(1) if has_batch_norm else None
    self.relu = nn.ReLU() if has_relu else None

    self.sigmoid = nn.Sigmoid()

  def forward(self, x):
    raw_input = x

    # compress input
    x = torch.cat( (torch.max(x,1)[0].unsqueeze(1), torch.mean(x,1).unsqueeze(1)), dim=1 )

    # conv layer
    x = self.conv(x)
    if self.bn is not None:
      x = self.bn(x)
    if self.relu is not None:
      x = self.relu(x)

    x = self.sigmoid(x)

    output = raw_input * x
    return output

class ChannelAttention(nn.Module):
  def __init__(self, in_channels: int, reduction_ratio: int =16, ):
    super().__init__()

    # according to the CBAM research paper
    # use a shared MLP for attention inference to save parameters, as both of aggregated channel
    # features lie in the same semantic embedding space.
    self.mlp = nn.Sequential(
        # input layer
        nn.Linear(in_channels, in_channels // reduction_ratio),
        nn.ReLU(),
        # hidden layer
        nn.LazyLinear(in_channels // reduction_ratio),
        nn.ReLU(), #use relu within the hidden layers to mitigate risk of vanishing gradient
        nn.LazyLinear(in_channels // reduction_ratio),
        nn.ReLU(),
        # output layer
        nn.Linear(in_channels // reduction_ratio, in_channels)
    )
    self.sigmoid = nn.Sigmoid()

  def forward(self, x):
    raw_input = x
    # Maxpool input
    max_pool = F.max_pool2d( x,
                            kernel_size=(x.size(2), x.size(3)),
                            stride=(x.size(2), x.size(3)))
    # feed through MLP
    max_pool = max_pool.view(max_pool.size(0), -1) #flatten ready for linear layers
    channel_att_max = self.mlp( max_pool )

    # AvgPool input
    avg_pool = F.avg_pool2d( x,
                            kernel_size=(x.size(2), x.size(3)),
                             stride=(x.size(2), x.size(3)))
    # feed through MLP
    avg_pool = avg_pool.view(avg_pool.size(0), -1) #flatten ready for linear layers
    channel_att_avg = self.mlp( avg_pool )

    # add MLP results
    attention_mask = channel_att_max + channel_att_avg

    # feed through sigmoid
    attention_mask = self.sigmoid(attention_mask)
    # unsqueeze to make dimension of attention mask compatible with raw input
    # fixes RuntimeError: The size of tensor a (136) must match the size of tensor b (32) at non-singleton dimension 3
    attention_mask = attention_mask.unsqueeze(-1).unsqueeze(-1)

    output = raw_input * attention_mask
    return output

In [34]:
class CBAM(nn.Module):
  def __init__(self,
               in_channels:int,
               reduction_ratio: int = 16,
               is_channel_att: bool = True,
               is_spatial_att: bool = True) -> None:
    super().__init__()
    self.channel_attention = ChannelAttention(in_channels, reduction_ratio) if is_channel_att else None
    self.spatial_attention = SpatialAttention() if is_spatial_att else None

  def forward(self, x):
    if self.channel_attention is not None:
      x = self.channel_attention(x)
    if self.spatial_attention is not None:
      x = self.spatial_attention(x)

    return x

In [35]:
class ScaledCBAM(nn.Module):
  def __init__(self, in_channels, scaling_parameter):
    super().__init__()
    self.cbam = CBAM(in_channels)
    self.scale = scaling_parameter

  def forward(self, x):
    return self.scale * self.cbam(x)


In [36]:
def initialise_resnet_block(num_filters: int,
                     kernel_size: int,
                     stride: int,
                     padding: int,
                     cbam_module: CBAM | ScaledCBAM = None
                     ) -> torch.nn.Sequential:
  """
  Initialises a ResNet block (consisting of one 'Conv block" and one "Identity block")

  Args:
  num_filters: the number of filters for both blocks
  kernel_size: the kernel size for both blocks
  stride: the stride value for the first block only (conv block)
          (identity block must have stride of 1 so that dimensions remain unchanged)
  padding: padding size for both conv blocks
  cbam_module (optional parameter): add CBAM attention module to the block - this parameter is an instantiation one of the 2 class types ^

  Returns: a PyTorch sequential container (effectively a model)

  """

  if cbam_module:
    return nn.Sequential(ConvBlock(num_filters=num_filters,
                                        kernel =kernel_size,
                                        stride_val=stride,
                                        input_padding =padding ),

                          IdentityBlock(num_filters=num_filters,
                                        kernel=kernel_size,
                                        input_padding=padding),
                         cbam_module)
  else:
    return nn.Sequential(ConvBlock(num_filters=num_filters,
                                    kernel =kernel_size,
                                    stride_val=stride,
                                    input_padding =padding ),

                          IdentityBlock(num_filters=num_filters,
                                        kernel=kernel_size,
                                        input_padding=padding))

### Definition of Feature Pyramid Network:

In [37]:
# the code in this cell is adapted from Kapil Sachdeva:
# https://www.youtube.com/watch?v=FKsgO0U7CUw&list=PLjRC2jqp3e7QNio7NJXRh_gvAdLV6_aZC&index=132
# the code has been modified so that it accomodates all 4 layers (feature maps) of the ResNetCBAM backbone (defined above)
from functools import partial
from torchvision.ops.misc import Conv2dNormActivation

# TOP DOWN FPN
class FPN(nn.Module):
  def __init__(self, in_channels: list[int], out_channels: int) -> list:
    """Implements top down feature pyramid network

      Sets the model to eval mode and runs the model to get inferences.
      Non-maximum suppression is applied before mAPs are calculated.
      mAP50 and mAP50-95 are printed for convenience
      Model predictions are returned so that they can be visualised by
      a follow up call to visualise_predictions()

      Args:
          in_channels: list of integers representing the number of channels for each feature map:
                        list should follow the order low level -> high level features
          out_channels: the number of channels each output feature map will have

      Returns:
          A list containing the resulting feature maps (ordered high level -> low level)

      Raises:
          None
      """
    super().__init__()

    LeakyRelu_Inplace = partial(
        nn.LeakyReLU,
        negative_slope=0.1,
        inplace=True,
    )

    ConvBlock = partial(Conv2dNormActivation, activation_layer=LeakyRelu_Inplace)

    self.output_channels = out_channels
    if in_channels[3] != out_channels:
      self.hl_channel_reducer = ConvBlock(
          in_channels=in_channels[3],
          out_channels=out_channels,
          kernel_size=1,
      )

    self.ml_1_channel_reducer = ConvBlock(
        in_channels=in_channels[2],
        out_channels=out_channels,
        kernel_size=1,
    )

    self.ml_2_channel_reducer = ConvBlock(
        in_channels=in_channels[1],
        out_channels=out_channels,
        kernel_size=1,
    )

    self.ll_channel_reducer = ConvBlock(
        in_channels=in_channels[0],
        out_channels=out_channels,
        kernel_size=1,
    )

    self.ml_1_smoother = ConvBlock(
        in_channels=out_channels,
        out_channels=out_channels,
    )

    self.ml_2_smoother = ConvBlock(
        in_channels=out_channels,
        out_channels=out_channels,
    )

    self.ll_smoother = ConvBlock(
        in_channels=out_channels,
        out_channels=out_channels,
    )

  def forward(self,
              hl_features:torch.Tensor,
              ml_1_features:torch.Tensor,
              ml_2_features:torch.Tensor,
              ll_features:torch.Tensor):

    if hl_features.size()[1] != self.output_channels:
      hl_channel_r = self.hl_channel_reducer(hl_features)
      del hl_features

    ml_channel_1_r = self.ml_1_channel_reducer(ml_1_features) #Given groups=1, weight of size [256, 128, 1, 1], expected input[16, 64, 50, 68] to have 128 channels, but got 64 channels instead
    del ml_1_features
    ml_channel_2_r = self.ml_2_channel_reducer(ml_2_features)
    del ml_2_features
    ll_channel_r = self.ll_channel_reducer(ll_features)
    del ll_features

    # addition 1
    if hl_features.size()[1] != self.output_channels:
      upsample_hl = F.interpolate(
          hl_channel_r,
          size=[ml_channel_1_r.size(2), ml_channel_1_r.size(3)],
          mode="nearest",
      )
    else:
      upsample_hl = F.interpolate(
          hl_features,
          size=[ml_channel_1_r.size(2), ml_channel_1_r.size(3)],
          mode="nearest",
      )

    fused_hl_ml_1 = upsample_hl + ml_channel_1_r
    smoothed_ml_1_features = self.ml_1_smoother(fused_hl_ml_1)
    # memory management
    del ml_channel_1_r, fused_hl_ml_1, upsample_hl

    # addition 2
    upsample_ml_1 = F.interpolate(
        smoothed_ml_1_features,
        size=[ml_channel_2_r.size(2), ml_channel_2_r.size(3)],
        mode="nearest",
    )
    fused_ml_2_ml_1 = upsample_ml_1 + ml_channel_2_r
    smoothed_ml_2_features = self.ml_2_smoother(fused_ml_2_ml_1)
    # memory management
    del upsample_ml_1, ml_channel_2_r, fused_ml_2_ml_1

    # addition 3
    upsample_ml_2 = F.interpolate(
        smoothed_ml_2_features,
        size=[ll_channel_r.size(2), ll_channel_r.size(3)],
        mode="nearest",
    )
    fused_ml_ll = upsample_ml_2 + ll_channel_r
    smoothed_ll_features = self.ll_smoother(fused_ml_ll)
    # memory management
    del upsample_ml_2, ll_channel_r, fused_ml_ll

    out = [smoothed_ll_features,
           smoothed_ml_1_features,
           smoothed_ml_2_features,
           hl_features if hl_features.size()[1] == self.output_channels else hl_channel_r,
           ]
    return out

In [ ]:
class ResNetCBAMCustomFPN(nn.Module):
    """"
    Initialises a custom network that is a blend of ResNet and CBAM Attention

    Attributes:
    - block 1 is created in line with definition of original ResNet research paper
    - blocks 2-4 are composed of: ResNet "conv block" + ResNet "Identity Block" + CBAM attention module
    - block 5 is composed of: ResNet "conv block" + ResNet "Identity Block"

    """
    def __init__(self,
                 input_channels: int =3, # datasets imgs are in colour so default input channels is 3
                 num_classes: int =4) -> None:

        """
        Initalises 5 blocks ready for the forward pass

        Args:
        input_channels: number of channles in the input image (3 for color imgs)
        num_classes: the number of classes in the dataset

        Returns: None
        """
        super().__init__()
        # a conv layer with kernel=3, stride=1, padding=1 is known as 'same convolution' meaning the output size won't change
        # first block of resnet is defined as follows
        self.block1 = nn.Sequential(
            Conv2d(in_channels=input_channels,
                      out_channels=16, #output channels = number of filters
                      kernel_size=7,
                      stride=2,
                      padding=3),

            nn.BatchNorm2d(16),
            LeakyReLU(),
            nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        )

        self.block2 = initialise_resnet_block(num_filters=32,
                                              kernel_size=3,
                                              stride=2,
                                              padding=1,
                                              cbam_module=ScaledCBAM(in_channels=32, scaling_parameter=nn.Parameter(torch.zeros(1))))

        self.block3 = initialise_resnet_block(num_filters=64,
                                              kernel_size=3,
                                              stride=2,
                                              padding=1,
                                              cbam_module=CBAM(in_channels=64))

        self.block4 = initialise_resnet_block(num_filters=128,
                                              kernel_size=3,
                                              stride=2,
                                              padding=1,
                                              cbam_module=CBAM(in_channels=128))

        self.block5 = initialise_resnet_block(num_filters=256,
                                              kernel_size=3,
                                              stride=2,
                                              padding=1)

        self.fpn = FPN(in_channels= [32,64,128,256], out_channels= 256)

        # TEST
        # self.dropout = nn.Dropout(p=0.25)

    def forward(self, x: torch.tensor) -> torch.tensor :
        x = self.block1(x)

        ll_feats = self.block2(x)
        ml_2_feats = self.block3(ll_feats)
        ml_1_feats = self.block4(ml_2_feats)
        hl_feats = self.block5(ml_1_feats)

        x = self.fpn(hl_features = hl_feats,
              ml_1_features= ml_1_feats,
              ml_2_features= ml_2_feats,
              ll_features= ll_feats)

        return OrderedDict([(k, v) for k, v in zip(["ll_feats",
                                                    "ml_1_feats",
                                                    "ml_2_feats",
                                                    "hl_feats"], x)])

In [ ]:
backbone_fpn = ResNetCBAMCustomFPN(num_classes=3)
backbone_fpn.out_channels = 256

# anchor_generator = AnchorGenerator(
#     sizes=((32, 64, 128, 256),),
#     aspect_ratios=((0.5, 1.0, 2.0),)
# )

anchor_sizes = ((16,), (32,), (64,), (128,))
aspect_ratios = (((0.5,), (1.0,), (2.0,)),) * len(anchor_sizes)
anchor_generator = AnchorGenerator(anchor_sizes, aspect_ratios)

# Feature maps to perform RoI cropping.
# If backbone returns a Tensor, `featmap_names` is expected to
# be [0]. We can choose which feature maps to use.
roi_pooler = torchvision.ops.MultiScaleRoIAlign(
    featmap_names=["ll_feats",
                  "ml_1_feats",
                  "ml_2_feats",
                  "hl_feats"],
    output_size=7, #output size for pooled region
    sampling_ratio=2 #sampling ratio for ROIAlign
)

fpn_model_wrapper = FasterRCNNWrapper(
    backbone=backbone_fpn,
    num_classes=3,
    rpn_anchor_generator=anchor_generator,
    box_roi_pool=roi_pooler
)
num_classes=3 #3 classes +1 to account for background
fpn_model_wrapper.override_prediction_heads(num_classes)

In [ ]:
fpn_model_wrapper.model.to(device)
# optimizer = torch.optim.Adam(cbam_model_wrapper.model.parameters(), lr=0.0005)
optimizer = torch.optim.Adam(fpn_model_wrapper.model.parameters(), lr=0.00025)
num_epochs = 15
model_pipeline(fpn_model_wrapper , optimizer, num_epochs, train_dataset_loader,
               val_dataset_loader, test_dataset_loader, device,
               True, "Custom Backbone Faster R-CNN Tracker")

### FPN With Dynamic texture attention

In [ ]:
import torch.nn as nn
from functools import partial
from torchvision.ops.misc import Conv2dNormActivation

class FPNDynamicTextureAttention(nn.Module):
  def __init__(self,
               ll_feat_input_size: int,
               hl_feat_input_size: int):

    super().__init__()


    LeakyRelu_Inplace = partial(
        nn.LeakyReLU,
        negative_slope=0.1,
        inplace=True,
    )

    ConvBlock = partial(Conv2dNormActivation, activation_layer=LeakyRelu_Inplace)


    # conv blocks to upsample and downsample as needed
    # tried using conv blocks here for up/downsamplers but made performance worse
    self.upsampler = nn.Conv2d(in_channels= 48,
                               out_channels=256,
                               kernel_size=3,
                               )

    self.downsampler = nn.Conv2d(in_channels= 256,
                               out_channels=48,
                               kernel_size=3

                             )

    self.intermediate_upsampler = nn.Conv2d(in_channels=32, out_channels=48, kernel_size=3)
    self.ll_final_upsampler = nn.Conv2d(in_channels=48, out_channels=256, kernel_size=3)

    # feature refiner for lowerl level features
    self.feature_refiner = nn.Sequential(
        nn.AvgPool2d(kernel_size=1),
        nn.LazyConv2d(out_channels=96, kernel_size=1),
        nn.ReLU(),
        nn.LazyConv2d(out_channels=96, kernel_size=1), #out_channels used to be ll_feat_input_size
        nn.Sigmoid()
    )

    self.dilated_conv_1 = nn.Conv2d(in_channels= ll_feat_input_size,
                                    out_channels=ll_feat_input_size,
                                    kernel_size=3,
                                    dilation=1,
                                    stride=1
                                  )

    self.dilated_conv_2 = nn.Conv2d(in_channels= ll_feat_input_size,
                                    out_channels=ll_feat_input_size,
                                    kernel_size=3,
                                    dilation=2,
                                    stride=1,
                                  )
    self.dilated_conv_3 = nn.Conv2d(in_channels= ll_feat_input_size,
                                    out_channels=ll_feat_input_size,
                                    kernel_size=3,
                                    dilation=3,
                                    stride=1,
                                  )

    self.sigmoid = nn.Sigmoid()

    # smoothers
    self.ll_smoother = ConvBlock(
        in_channels=96,
        out_channels=96,
    )

    self.ll_multiplied_smoother = ConvBlock(
        in_channels=96,
        out_channels=96,
    )

    self.ll_added_smoother = ConvBlock(
        in_channels=96,
        out_channels=96,
    )

    self.refined_ll_smoother = ConvBlock(
        in_channels=48,
        out_channels=48,
    )

    self.intermediate_feats_smoother = ConvBlock(
        in_channels=256,
        out_channels=256,
    )

    self.refined_hl_smoother = ConvBlock(
        in_channels=256,
        out_channels=256,
    )

    self.hl_ll_multiplied_smoother = ConvBlock(
        in_channels=48,
        out_channels=48,
    )

    self.hl_ll_subtracted_smoother = ConvBlock(
        in_channels=48,
        out_channels=48,
    )

  def forward(self, ll_feat_map, hl_feat_map):

    ll_dilated_1 = self.dilated_conv_1(ll_feat_map)
    ll_dilated_2 = self.dilated_conv_2(ll_feat_map)
    ll_dilated_3 = self.dilated_conv_3(ll_feat_map)
    # increase height and width of ll_dilated_2 and 3 so that they can be concatenated with ll_dilated_1
    upsampled_ll_dilated_2 = F.interpolate(
        ll_dilated_2,
        size=[ll_dilated_1.size(2), ll_dilated_1.size(3)],
        mode="nearest",
    )
    upsampled_ll_dilated_3 = F.interpolate(
        ll_dilated_3,
        size=[ll_dilated_1.size(2), ll_dilated_1.size(3)],
        mode="nearest",
    )

    #first dimension is batch size
    ll_x = torch.cat((ll_dilated_1, upsampled_ll_dilated_2, upsampled_ll_dilated_3), dim=1) #is dim 1 ?
    # smoothes ll_x
    ll_x = self.ll_smoother(ll_x)

    ll_x_refined = self.feature_refiner(ll_x)
    ll_x_combined = ll_x * ll_x_refined
    #smoothing out artificats introduced by F.interpolate
    ll_x_combined = self.ll_multiplied_smoother(ll_x_combined)

    #split & sum
    half_1, half_2 = torch.split(ll_x_combined, ll_x_combined.size(dim=1)//2, dim=1)
    ll_combined_feats = half_1 + half_2
    #smoothing out artificats introduced by F.interpolate
    ll_x_combined = self.ll_added_smoother(ll_x_combined)

    upsampled_ll_combined = F.interpolate(
        ll_combined_feats,
        size=[ll_feat_map.size(2), ll_feat_map.size(3)],
        mode="nearest",
    )

    upsampled_ll_feat_map = self.intermediate_upsampler(ll_feat_map)
    upsampled_ll_feat_map = F.interpolate(
        upsampled_ll_feat_map,
        size=[upsampled_ll_combined.size(2), upsampled_ll_combined.size(3)],
        mode="nearest",
    )
    refined_ll_feat_map = upsampled_ll_feat_map * upsampled_ll_combined
    refined_ll_feat_map = self.refined_ll_smoother(refined_ll_feat_map)

    upsampled_hl = F.interpolate(
        hl_feat_map,
        size=[refined_ll_feat_map.size(2), refined_ll_feat_map.size(3)],
        mode="nearest",
    )

    usampled_hl = self.sigmoid(upsampled_hl)
    upsampled_refined_ll = self.upsampler(refined_ll_feat_map)

    # upsample height and width of conv output
    upsampled_refined_ll = F.interpolate(
        upsampled_refined_ll,
        size=[upsampled_hl.size(2), upsampled_hl.size(3)],
        mode="nearest",
    )
    intermediate_feat_map = upsampled_refined_ll * upsampled_hl
    intermediate_feat_map = self.intermediate_feats_smoother(intermediate_feat_map)

    downsampled_intermediate_feats = F.interpolate(
        intermediate_feat_map,
        size=[hl_feat_map.size(2), hl_feat_map.size(3)],
        mode="nearest",
    )
    refined_hl_feat_map = downsampled_intermediate_feats + hl_feat_map
    refined_hl_feat_map = self.refined_hl_smoother(refined_hl_feat_map)

    # downsample ready for element wise multiplication
    downsampled_intermediate_feats = self.downsampler(intermediate_feat_map)

    downsampled_intermediate_feats = F.interpolate(
        downsampled_intermediate_feats,
        size=[refined_ll_feat_map.size(2), refined_ll_feat_map.size(3)],
        mode="nearest",
    )
    ll_hl_combined_feat_map = refined_ll_feat_map * downsampled_intermediate_feats
    ll_hl_combined_feat_map = self.hl_ll_multiplied_smoother(ll_hl_combined_feat_map)

    subtracted_ll_feat_map = refined_ll_feat_map - ll_hl_combined_feat_map
    subtracted_ll_feat_map = self.hl_ll_subtracted_smoother(subtracted_ll_feat_map)

    #upsamople the subtracted_ll_feat_map so that it can be processed by the RPN
    # RPN requires all feature maps to have the same number of channels
    final_ll_feat_map = self.ll_final_upsampler(subtracted_ll_feat_map)
    return [final_ll_feat_map, refined_hl_feat_map]

In [ ]:
class ResNetCustomFPNWithAtt(nn.Module):
    """"
    Initialises a custom network that is a blend of ResNet and CBAM Attention

    Attributes:
    - block 1 is created in line with definition of original ResNet research paper
    - blocks 2-4 are composed of: ResNet "conv block" + ResNet "Identity Block" + CBAM attention module
    - block 5 is composed of: ResNet "conv block" + ResNet "Identity Block"

    """
    def __init__(self,
                 input_channels: int =3, # datasets imgs are in colour so default input channels is 3
                 num_classes: int =4) -> None:

        """
        Initalises 5 blocks ready for the forward pass

        Args:
        input_channels: number of channles in the input image (3 for color imgs)
        num_classes: the number of classes in the dataset

        Returns: None
        """
        super().__init__()
        # a conv layer with kernel=3, stride=1, padding=1 is known as 'same convolution' meaning the output size won't change
        # first block of resnet is defined as follows
        self.block1 = nn.Sequential(
            Conv2d(in_channels=input_channels,
                      out_channels=16, #output channels = number of filters
                      kernel_size=7,
                      stride=2,
                      padding=3),

            nn.BatchNorm2d(16),
            LeakyReLU(),
            nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        )

        self.block2 = initialise_resnet_block(num_filters=32,
                                              kernel_size=3,
                                              stride=2,
                                              padding=1,
                                              cbam_module=ScaledCBAM(in_channels=32, scaling_parameter=nn.Parameter(torch.zeros(1))))

        self.block3 = initialise_resnet_block(num_filters=64,
                                              kernel_size=3,
                                              stride=2,
                                              padding=1,
                                              cbam_module=CBAM(in_channels=64))

        self.block4 = initialise_resnet_block(num_filters=128,
                                              kernel_size=3,
                                              stride=2,
                                              padding=1,
                                              cbam_module=CBAM(in_channels=128))

        self.block5 = initialise_resnet_block(num_filters=256,
                                              kernel_size=3,
                                              stride=2,
                                              padding=1)

        self.fpn = FPNDynamicTextureAttention(ll_feat_input_size = 32, hl_feat_input_size=256) #should probably have output_channels =256

        self.ml_1_upsampler = nn.Conv2d(in_channels=64, out_channels=256, kernel_size=3)
        self.ml_2_upsampler = nn.Conv2d(in_channels=128, out_channels=256, kernel_size=3)

        # TEST
        # self.dropout = nn.Dropout(p=0.25)

    def forward(self, x: torch.tensor) -> torch.tensor :

        # test = x.view(x.size(0), -1)
        # print('block 1 input shape')
        # print(test.shape)

        x = self.block1(x)


        # x = self.block2(x)
        ll_feats = self.block2(x)


        # x = self.block3(x)
        ml_1_feats = self.block3(ll_feats)


        # x = self.block4(x)
        ml_2_feats = self.block4(ml_1_feats)

        # x = self.block5(x)
        hl_feats = self.block5(ml_2_feats)

        x = self.fpn(ll_feat_map = ll_feats, hl_feat_map = hl_feats)

        ml_1_upsampled = self.ml_1_upsampler(ml_1_feats)
        ml_2_upsampled = self.ml_2_upsampler(ml_2_feats)
        feat_maps = [x[0], ml_1_upsampled, ml_2_upsampled, x[1]]


        # for item in feat_maps:
        #   print(item.size())

        # torch.Size([16, 256, 98, 134])
        # torch.Size([16, 256, 48, 66])
        # torch.Size([16, 256, 23, 32])
        # torch.Size([16, 256, 13, 17])

        # torch.Size([16, 256, 98, 134])
        # torch.Size([16, 64, 50, 68])
        # torch.Size([16, 128, 25, 34])
        # torch.Size([16, 256, 100, 136])

        return OrderedDict([(k, v) for k, v in zip(["ll_feats",
                                                    "ml_1_feats",
                                                    "ml_2_feats",
                                                    "hl_feats"
                                                    ], feat_maps)])

In [ ]:
backbone_fpn = ResNetCustomFPNWithAtt(num_classes=3)
#specify outchannels as it used as inchannels for the RPNHeads
backbone_fpn.out_channels = 256

anchor_generator = AnchorGenerator(
    sizes=((32, 64, 128, 256),),
    aspect_ratios=((0.5, 1.0, 2.0),)
)

anchor_sizes = ((16,), (32,), (64,), (128,))
aspect_ratios = (((0.5,), (1.0,), (2.0,)),) * len(anchor_sizes)
anchor_generator = AnchorGenerator(anchor_sizes, aspect_ratios)

# debugging
anchor_sizes = ((16,), (32,), (64,), (128,))
aspect_ratios = (((0.5,), (1.0,), (2.0,)),) * len(anchor_sizes)
anchor_generator = AnchorGenerator(anchor_sizes, aspect_ratios)

# Feature maps to perform RoI cropping.
# If backbone returns a Tensor, `featmap_names` is expected to
# be [0]. We can choose which feature maps to use.
roi_pooler = torchvision.ops.MultiScaleRoIAlign(
    featmap_names=["ll_feats",
                  "ml_1_feats",
                  "ml_2_feats",
                  "hl_feats"],
    output_size=7, #output size for pooled region
    sampling_ratio=2 #sampling ratio for ROIAlign
)

fpn_att_model_wrapper = FasterRCNNWrapper(
    backbone=backbone_fpn,
    num_classes=3,
    rpn_anchor_generator=anchor_generator,
    box_roi_pool=roi_pooler
)
num_classes=3 #3 classes +1 to account for background
fpn_att_model_wrapper.override_prediction_heads(num_classes)

In [ ]:
fpn_att_model_wrapper.model.to(device)
# optimizer = torch.optim.Adam(cbam_model_wrapper.model.parameters(), lr=0.0005)
optimizer = torch.optim.Adam(fpn_att_model_wrapper.model.parameters(), lr=0.00025)
num_epochs = 15
model_pipeline(fpn_att_model_wrapper , optimizer, num_epochs, train_dataset_loader,
               val_dataset_loader, test_dataset_loader, device,
               False, "Custom Backbone Faster R-CNN Tracker")

In [ ]:
# fpn_DTA_weights_path = '/content/drive/MyDrive/Colab Notebooks/UoL FP/my_data_model_test/saved_models/custom_fpn_with_DTA_model_weights.pth'
# torch.save(fpn_att_model_wrapper.model.state_dict(), fpn_DTA_weights_path)

In [ ]:
# model_robustness_curve(anchor_generator=anchor_generator,
#                        roi_pooler=roi_pooler,
#                        backbone=backbone_fpn,
#                        train_dataset=train_dataset,
#                        list_dataset_subsamples=[0.5,0.8,1],
#                        num_epochs=5,
#                        val_dataset_loader=val_dataset_loader,
#                        test_dataset_loader=test_dataset_loader)

## Custom Backbone 2: Biomimetic Blocks + CBAM + FPN:

In [41]:
import torch.nn as nn

class ShallowFovea(nn.Module):
  def __init__(self, num_channels: int, out_channels: int):
    """Implements the Shallow Fovea Module as proposed by Chen et Lin

      Implements the biomimetic convolutional block introduced by the research paper:
      https://www.sciencedirect.com/science/article/abs/pii/S1051200424005815
      This block mimics the visual pathways in eagle eyes in order to detect small
      objects in aerial imagery. The large kernel sizes of this module simulate the
      wide field of view of an eagle.
      This module utilises branches (similar to the Inception model)
      Batch normalisation and activation functions are added to each conv layer
      using a combination of functools.parital() and torch.Conv2dNormActivation.

      Args:
          num_channels: number of channels for the first layer
          out_channels: the number of channels each output feature map will have

      Returns:
          a torch tensor

      Raises:
          None
      """
    super().__init__()

    LeakyRelu_Inplace = partial(
        nn.LeakyReLU,
        negative_slope=0.1,
        inplace=True,
    )

    ConvBlock = partial(Conv2dNormActivation, activation_layer=LeakyRelu_Inplace)

    self.branch_1 = nn.Sequential(
        ConvBlock(in_channels=num_channels, out_channels=num_channels, kernel_size=1),
        ConvBlock(in_channels=num_channels, out_channels=num_channels, kernel_size=7),
    )

    self.branch_2 = nn.Sequential(
        ConvBlock(in_channels=num_channels, out_channels=num_channels, kernel_size=1),
        ConvBlock(in_channels=num_channels, out_channels=num_channels, kernel_size=(1,7)),
        ConvBlock(in_channels=num_channels, out_channels=num_channels, kernel_size=(7,1)),
        ConvBlock(in_channels=num_channels, out_channels=num_channels, kernel_size=7, dilation=3),
    )

    self.branch_3 = nn.Sequential(
        ConvBlock(in_channels=num_channels, out_channels=num_channels, kernel_size=1),
        ConvBlock(in_channels=num_channels, out_channels=num_channels, kernel_size=(3,1)),
        ConvBlock(in_channels=num_channels, out_channels=num_channels, kernel_size=(1,3)),
        ConvBlock(in_channels=num_channels, out_channels=num_channels, kernel_size=7, dilation=3),
    )

    self.branch_4 = nn.Sequential(
        ConvBlock(in_channels=num_channels, out_channels=out_channels, kernel_size=1),
    )

    self.downsampler = nn.LazyConv2d(out_channels=out_channels, kernel_size=3)


  def forward(self, x):
    branch_1 = self.branch_1(x)
    branch_2 = self.branch_2(x)
    branch_3 = self.branch_3(x)
    branch_4 = self.branch_4(x)

    upsampled_branch_1 = F.interpolate(
        branch_1,
        size=[branch_3.size(2), branch_3.size(3)],
        mode="bilinear",
    )
    del branch_1 #remove intermediate calculation from GPU RAM clear
    upsampled_branch_2 = F.interpolate(
        branch_2,
        size=[branch_3.size(2), branch_3.size(3)],
        mode="bilinear",
    )
    del branch_2 #remove intermediate calculation from GPU RAM clear

    branch_concat = torch.cat( (upsampled_branch_1,
                                upsampled_branch_2,
                                branch_3), dim=1 ) #concat on first dimension to allow for different number of channels
    del upsampled_branch_1, upsampled_branch_2, branch_3


    # prep ready for element wise multiplication
    downsampled_branch_concat = self.downsampler(branch_concat)
    del branch_concat #remove intermediate calculation from GPU RAM clear

    resized_branch_concat = F.interpolate(
        downsampled_branch_concat,
        size=[branch_4.size(2), branch_4.size(3)],
        mode="bilinear",
    )
    result = branch_4 * resized_branch_concat
    del branch_4, resized_branch_concat
    return result

In [42]:
import torch.nn as nn

class DeepFovea(nn.Module):
  def __init__(self, num_channels: int, out_channels: int):
    """Implements the Deep Fovea Module as proposed by Chen et Lin

      Implements the biomimetic convolutional block introduced by the research paper:
      https://www.sciencedirect.com/science/article/abs/pii/S1051200424005815
      This block mimics the visual pathways in eagle eyes in order to detect small
      objects in aerial imagery. The small kernel sizes of this module preserve the
      pixel data of small objects - effectively simulating the magnification mechanism
      in eagle eyes which allows the to spot small prey during flight.
      This module utilises branches (similar to the Inception model)
      Batch normalisation and activation functions are added to each conv layer
      using a combination of functools.parital() and torch.Conv2dNormActivation.

      Args:
          num_channels: number of channels for the first layer
          out_channels: the number of channels each output feature map will have

      Returns:
          a torch tensor

      Raises:
          None
      """
    super().__init__()

    LeakyRelu_Inplace = partial(
        nn.LeakyReLU,
        negative_slope=0.1,
        inplace=True,
    )

    ConvBlock = partial(Conv2dNormActivation, activation_layer=LeakyRelu_Inplace)

    self.branch_1 = nn.Sequential(
        ConvBlock(in_channels=num_channels, out_channels=num_channels, kernel_size=1),
        ConvBlock(in_channels=num_channels, out_channels=num_channels, kernel_size=3),
    )

    self.branch_2 = nn.Sequential(
        ConvBlock(in_channels=num_channels, out_channels=num_channels, kernel_size=1),
        ConvBlock(in_channels=num_channels, out_channels=num_channels, kernel_size=(1,3)),
        ConvBlock(in_channels=num_channels, out_channels=num_channels, kernel_size=(3,1)),
        ConvBlock(in_channels=num_channels, out_channels=num_channels, kernel_size=3, dilation=3),
    )

    self.branch_3 = nn.Sequential(
        ConvBlock(in_channels=num_channels, out_channels=num_channels, kernel_size=1),
        ConvBlock(in_channels=num_channels, out_channels=num_channels, kernel_size=(3,1)),
        ConvBlock(in_channels=num_channels, out_channels=num_channels, kernel_size=(1,3)),
        ConvBlock(in_channels=num_channels, out_channels=num_channels, kernel_size=3, dilation=3),
    )

    self.branch_4 = nn.Sequential(
        ConvBlock(in_channels=num_channels, out_channels=out_channels, kernel_size=1),
    )

    self.downsampler = nn.LazyConv2d(out_channels=out_channels, kernel_size=3)


  def forward(self, x):
    branch_1 = self.branch_1(x)
    branch_2 = self.branch_2(x)
    branch_3 = self.branch_3(x)
    branch_4 = self.branch_4(x)

    upsampled_branch_1 = F.interpolate(
        branch_1,
        size=[branch_3.size(2), branch_3.size(3)],
        mode="bilinear",
    )
    del branch_1 #remove intermediate calculation from GPU RAM clear
    upsampled_branch_2 = F.interpolate(
        branch_2,
        size=[branch_3.size(2), branch_3.size(3)],
        mode="bilinear",
    )
    del branch_2 #remove intermediate calculation from GPU RAM clear

    branch_concat = torch.cat( (upsampled_branch_1,
                                upsampled_branch_2,
                                branch_3), dim=1 ) #concat on first dimension to allow for different number of channels
    del upsampled_branch_1, upsampled_branch_2, branch_3

    # prep ready for element wise multiplication
    downsampled_branch_concat = self.downsampler(branch_concat)
    del branch_concat #remove intermediate calculation from GPU RAM clear

    resized_branch_concat = F.interpolate(
        downsampled_branch_concat,
        size=[branch_4.size(2), branch_4.size(3)],
        mode="bilinear",
    )

    result = branch_4 * resized_branch_concat
    del branch_4, resized_branch_concat
    return result

In [43]:
# TO DO:
# - experiment with residual connections
from functools import partial
from torchvision.ops import Conv2dNormActivation

class FoveaBackbone(nn.Module):
  def __init__(self, in_channels: int):
    """Integrates ShallowFovea and DeepFovea modules, CBAM attention and an FPN into
        a backbone ready to be integrated into Faster R-CNN

      Each block of (ShallowFovea, DualFovea) is followed by a CBAM attention mechanism
      (Convolutional Block Attention Module). The feature maps produced at each stage
      in the network are then fed through a top-down FPN (Feature Pyramid Network)
      which infuses the lower level features with the semantic information extract
      from deeper convolutional layers.

      Args:
          in_channels: the number of channels to be passed to the first block

      Returns:
          a list of feature maps (ready to be passed to the Faster R-CNN Region
          Proposal Network)

      Raises:
          None
      """
    super().__init__()

    LeakyRelu_Inplace = partial(
        nn.LeakyReLU,
        negative_slope=0.1,
        inplace=True,
    )

    ConvBlock = partial(Conv2dNormActivation, activation_layer=LeakyRelu_Inplace)

    #stride not mention in EVM paper but needed for memory management
    self.block_1 = nn.Sequential(
        ConvBlock(in_channels=in_channels, out_channels=32, kernel_size=3, stride=2),
        nn.MaxPool2d(kernel_size=2)
    )
    self.shallow_fovea_1 = ShallowFovea(num_channels=32, out_channels=64)
    self.deep_fovea_1 = DeepFovea(num_channels=32, out_channels=64)

    self.shallow_fovea_2 = ShallowFovea(num_channels=64, out_channels=128)
    self.deep_fovea_2 = DeepFovea(num_channels=64, out_channels=128)

    self.shallow_fovea_3 = ShallowFovea(num_channels=128, out_channels=256)
    self.deep_fovea_3 = DeepFovea(num_channels=128, out_channels=256)

    self.shallow_fovea_4 = ShallowFovea(num_channels=256, out_channels=512)
    self.deep_fovea_4 = DeepFovea(num_channels=256, out_channels=512)

    self.max_pooler = nn.MaxPool2d(kernel_size=2)
    self.fpn = FPN(in_channels=[64, 128, 256, 512], out_channels=512)
    # self.fpn = FPNDynamicTextureAttention(ll_feat_input_size=64, hl_feat_input_size=512)

    self.cbam_scaler_1 = nn.Parameter(torch.zeros(1))
    self.cbam_1 = ScaledCBAM(in_channels=64, scaling_parameter=self.cbam_scaler_1)
    self.cbam_scaler_2 = nn.Parameter(torch.zeros(1))
    self.cbam_2 = ScaledCBAM(in_channels=128, scaling_parameter=self.cbam_scaler_2)
    self.cbam_scaler_3 = nn.Parameter(torch.zeros(1))
    self.cbam_3 = ScaledCBAM(in_channels=256, scaling_parameter=self.cbam_scaler_3)
    self.cbam_scaler_4 = nn.Parameter(torch.zeros(1))
    self.cbam_4 = ScaledCBAM(in_channels=512, scaling_parameter=self.cbam_scaler_4)

  def forward(self, x):
    x = self.block_1(x)

    shallow_feats_1 = self.shallow_fovea_1(x)
    deep_feats_1 = self.deep_fovea_1(x)
    stage_1_results = shallow_feats_1 + deep_feats_1
    stage_1_results_pooled = self.max_pooler(stage_1_results) #reduce computational load for successive layers
    stage_1_results_w_cbam = self.cbam_1(stage_1_results)
    # memory management
    del shallow_feats_1, deep_feats_1, stage_1_results

    shallow_feats_2 = self.shallow_fovea_2(stage_1_results_pooled)
    deep_feats_2 = self.deep_fovea_2(stage_1_results_pooled)
    stage_2_results = shallow_feats_2 + deep_feats_2
    stage_2_results_pooled = self.max_pooler(stage_2_results) #reduce computational load for successive layers
    stage_2_results_w_cbam = self.cbam_2(stage_2_results)
    # memory management
    del shallow_feats_2, deep_feats_2, stage_2_results, stage_1_results_pooled,

    shallow_feats_3 = self.shallow_fovea_3(stage_2_results_pooled)
    deep_feats_3 = self.deep_fovea_3(stage_2_results_pooled)
    stage_3_results = shallow_feats_3 + deep_feats_3
    stage_3_results_pooled = self.max_pooler(stage_3_results) #reduce computational load for successive layers
    stage_3_results_w_cbam = self.cbam_3(stage_3_results)
    # memory management
    del shallow_feats_3, deep_feats_3, stage_3_results, stage_2_results_pooled,

    shallow_feats_4 = self.shallow_fovea_4(stage_3_results_pooled)
    deep_feats_4 = self.deep_fovea_4(stage_3_results_pooled)
    stage_4_results = shallow_feats_4 + deep_feats_4
    stage_4_results_w_cbam = self.cbam_4(stage_4_results)
    # memory management
    del shallow_feats_4, deep_feats_4, stage_4_results, stage_3_results_pooled,

    feat_maps = self.fpn(hl_features = stage_4_results_w_cbam,
                        ml_1_features= stage_3_results_w_cbam,
                        ml_2_features = stage_2_results_w_cbam,
                        ll_features= stage_1_results_w_cbam)

    return OrderedDict([(k, v) for k, v in zip(["ll_feats",
                                                "ml_1_feats",
                                                "ml_2_feats",
                                                "hl_feats"], feat_maps)])

In [44]:
import torchvision
from torchvision.models.detection import FasterRCNN
from torchvision.models.detection.rpn import AnchorGenerator

backbone = FoveaBackbone(3)
backbone.out_channels = 512

anchor_sizes = ((8,), (32,), (64,), (128,))
aspect_ratios = (((0.5,), (1.0,), (2.0,)),) * len(anchor_sizes)
anchor_generator = AnchorGenerator(anchor_sizes, aspect_ratios)

roi_pooler = torchvision.ops.MultiScaleRoIAlign(
    # specify the feature maps to be used by the region proposal network
    featmap_names=["ll_feats",
                    "ml_1_feats",
                    "ml_2_feats",
                    "hl_feats"],
    output_size=7,
    sampling_ratio=2
)

fovea_model_wrapper = FasterRCNNWrapper(
    backbone=backbone,
    num_classes=3,
    rpn_anchor_generator=anchor_generator,
    box_roi_pool=roi_pooler
)
num_classes=3 #3 classes +1 to account for background
fovea_model_wrapper.override_prediction_heads(num_classes)

In [ ]:
fovea_model_wrapper.model.to(device)
# optimizer = torch.optim.SGD(model.model.parameters() , lr=0.0005, momentum=0.9, weight_decay=0.0005)
optimizer = torch.optim.Adam(fovea_model_wrapper.model.parameters(), lr=0.0005)
num_epochs = 15
model_pipeline(fovea_model_wrapper , optimizer, num_epochs, train_dataset_loader,
               val_dataset_loader, test_dataset_loader, device,
               True, "Fovea Tracker")

## Cross validation

In [ ]:
backbone = ResNetCBAM(num_classes=3)
backbone.out_channels = 256

anchor_generator = AnchorGenerator(
    sizes=((32, 64, 128, 256, 512),),
    aspect_ratios=((0.5, 1.0, 2.0),)
)
# Feature maps to perform RoI cropping.
# If backbone returns a Tensor, `featmap_names` is expected to
# be [0]. We can choose which feature maps to use.
roi_pooler = torchvision.ops.MultiScaleRoIAlign(
    featmap_names=['0'],
    output_size=7, #output size for pooled region
    sampling_ratio=2 #sampling ratio for ROIAlign
)

k_fold_results = k_fold_training(train_dataset=train_dataset,
                                 n_splits=3,
                                 random_state=3,
                                 batch_size=16,
                                 backbone=backbone,
                                 anchor_generator=anchor_generator,
                                 roi_pooler=roi_pooler,
                                 num_epochs=15)

## Export Biomimetic backbone (Custom backbone 2) to ONNX format

In [ ]:
!pip install --upgrade onnx onnxscript

In [ ]:
sample_img = train_dataset[0][0]
backbone = FoveaBackbone(3)
backbone(sample_img)
onnx_program = torch.onnx.export(backbone, sample_img, dynamo=True)

## Pre-train custom backbone on large-scale dataset:

### Pre-training on the SODA-A Dataset:

In [38]:
from torch.utils.data import random_split
SODA_dataset = UAVImageDataset(data_dir_imgs="/content/drive/MyDrive/UoL Final Project/Project Data/Data for pre-training/SODA-A resized/resized_using_fx",
                                df_path="/content/drive/MyDrive/UoL Final Project/Project Data/Data for pre-training/SODA-A resized/SODA_pd.csv",
                                classes_start_at_zero=False)

SODA_dataset = UAVImageDataset(data_dir_imgs="/content/drive/MyDrive/UoL Final Project/Project Data/Data for pre-training/SODA-A resized/smallest_resizing",
                                df_path="/content/drive/MyDrive/UoL Final Project/Project Data/Data for pre-training/SODA-A resized/SODA_pd.csv",
                                classes_start_at_zero=False)

SODA_train_dataset, SODA_val_dataset, SODA_test_dataset = random_split(SODA_dataset, [0.8, 0.1, 0.1])

In [39]:
SODA_train_dataset_loader = torch.utils.data.DataLoader(SODA_train_dataset,
                                                   collate_fn=lambda x: x,
                                                   batch_size=6,
                                                   shuffle=True,
                                                  #  pin_memory=True,
                                                  # num_workers=2,
                                                  #  prefetch_factor=2
                                                   )

SODA_val_dataset_loader = torch.utils.data.DataLoader(SODA_val_dataset, collate_fn=lambda x: x, batch_size=6,
                                                  ) #num_workers=2

SODA_test_dataset_loader = torch.utils.data.DataLoader(SODA_test_dataset, collate_fn=lambda x: x, batch_size=6,
                                                  ) #num_workers=2

In [45]:
import torchvision
from torchvision.models.detection.rpn import AnchorGenerator

backbone = FoveaBackbone(3)
backbone.out_channels = 512

anchor_sizes = ((8,), (32,), (64,), (128,))
aspect_ratios = (((0.5,), (1.0,), (2.0,)),) * len(anchor_sizes)
anchor_generator = AnchorGenerator(anchor_sizes, aspect_ratios)

roi_pooler = torchvision.ops.MultiScaleRoIAlign(
    # specify the feature maps to be used by the region proposal network
    featmap_names=["ll_feats",
                    "ml_1_feats",
                    "ml_2_feats",
                    "hl_feats"],
    output_size=7,
    sampling_ratio=2
)

SODA_model_wrapper = FasterRCNNWrapper(
    backbone=backbone,
    num_classes=10, #SODA-A dataset has 9 classes (exlcuding the 'ignore class')
    rpn_anchor_generator=anchor_generator,
    box_roi_pool=roi_pooler
)
num_classes=10 #9 classes +1 to account for background
SODA_model_wrapper.override_prediction_heads(num_classes)

In [46]:
import time

SODA_model_wrapper.model.to(device)
# optimizer = torch.optim.SGD(model.model.parameters() , lr=0.0005, momentum=0.9, weight_decay=0.0005)
optimizer = torch.optim.Adam(SODA_model_wrapper.model.parameters(), lr=0.0005)
num_epochs = 5


start = time.time()
# model_pipeline(SODA_model_wrapper , optimizer, num_epochs, SODA_train_dataset_loader,
#                SODA_val_dataset_loader, SODA_test_dataset_loader, device,
#                False, "Fovea Tracker")

SODA_model_wrapper.train(num_epochs=15,
                         optimizer=optimizer,
                         train_dataset=SODA_train_dataset_loader,
                         is_console_logging=True,
                         calc_running_loss=False)
end = time.time()
print(end - start)

Epoch 0 of 15
batch complete
batch complete
batch complete
batch complete
batch complete
batch complete
batch complete
batch complete
batch complete
batch complete
batch complete
batch complete
batch complete
batch complete
batch complete
batch complete
batch complete
batch complete


OutOfMemoryError: CUDA out of memory. Tried to allocate 7.65 GiB. GPU 0 has a total capacity of 39.56 GiB of which 6.70 GiB is free. Process 2909 has 32.85 GiB memory in use. Of the allocated memory 31.26 GiB is allocated by PyTorch, and 278.55 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [ ]:
# save weights
SODA_DOTA_pretrained_weights_path = '/content/drive/MyDrive/Colab Notebooks/UoL FP/my_data_model_test/saved_models/SODA_pretrained_weights.pth'
torch.save(SODA_model_wrapper.model.state_dict(), SODA_DOTA_pretrained_weights_path)

In [ ]:
from google.colab import runtime
runtime.unassign()

### UAVVaste

In [47]:
# !pip install roboflow supervision --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.5/84.5 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.8/66.8 kB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.9/49.9 MB 40.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 181.5/181.5 kB 17.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 118.5 MB/s eta 0:00:00


In [ ]:
# /content/drive/MyDrive/UoL Final Project/Project Data/Data for pre-training/UAVVaste - prertraining.v1i.yolov8/test

### Pre-training on the DOTA dataset:

In [ ]:
# DOTA
DOTA_train_dataset = UAVImageDataset(data_dir_imgs="/content/drive/MyDrive/UoL Final Project/Project Data/Data for pre-training/DOTA formatted/train/images",
                                df_path="/content/drive/MyDrive/UoL Final Project/Project Data/Data for pre-training/DOTA formatted/DOTA_train_pd.csv",
                                classes_start_at_zero=False)

DOTA_val_dataset = UAVImageDataset(data_dir_imgs="/content/drive/MyDrive/UoL Final Project/Project Data/Data for pre-training/DOTA formatted/valid/images",
                                df_path="/content/drive/MyDrive/UoL Final Project/Project Data/Data for pre-training/DOTA formatted/DOTA_valid_pd.csv",
                              classes_start_at_zero=False)

DOTA_test_dataset = UAVImageDataset(data_dir_imgs="/content/drive/MyDrive/UoL Final Project/Project Data/Data for pre-training/DOTA formatted/test/images",
                                df_path="/content/drive/MyDrive/UoL Final Project/Project Data/Data for pre-training/DOTA formatted/DOTA_test_pd.csv",
                               classes_start_at_zero=False)

In [ ]:
DOTA_train_dataset_loader = torch.utils.data.DataLoader(DOTA_train_dataset,
                                                   collate_fn=lambda x: x,
                                                   batch_size=4, #was 8 for inception experiments
                                                   shuffle=True,
                                                  #  pin_memory=True,
                                                  num_workers=2,
                                                  #  prefetch_factor=2
                                                   )


DOTA_val_dataset_loader = torch.utils.data.DataLoader(DOTA_val_dataset, collate_fn=lambda x: x, batch_size=4, #was 8 for inception experiments
                                                  num_workers=2)

DOTA_test_dataset_loader = torch.utils.data.DataLoader(DOTA_test_dataset, collate_fn=lambda x: x, batch_size=4, #was 8 for inception experiments
                                                  num_workers=2)

In [ ]:
import torchvision
from torchvision.models.detection.rpn import AnchorGenerator

backbone = FoveaBackbone(3)
backbone.out_channels = 512

anchor_sizes = ((8,), (32,), (64,), (128,))
aspect_ratios = (((0.5,), (1.0,), (2.0,)),) * len(anchor_sizes)
anchor_generator = AnchorGenerator(anchor_sizes, aspect_ratios)

roi_pooler = torchvision.ops.MultiScaleRoIAlign(
    # specify the feature maps to be used by the region proposal network
    featmap_names=["ll_feats",
                    "ml_1_feats",
                    "ml_2_feats",
                    "hl_feats"],
    output_size=7,
    sampling_ratio=2
)

DOTA_model_wrapper = FasterRCNNWrapper(
    backbone=backbone,
    num_classes=17, #DOTA dataset has 16 classes (exlcuding the 'ignore class')
    rpn_anchor_generator=anchor_generator,
    box_roi_pool=roi_pooler
)
num_classes=17 #16 classes +1 to account for background
DOTA_model_wrapper.override_prediction_heads(num_classes)

In [ ]:
import time

DOTA_model_wrapper.model.to(device)
# optimizer = torch.optim.SGD(model.model.parameters() , lr=0.0005, momentum=0.9, weight_decay=0.0005)
optimizer = torch.optim.Adam(DOTA_model_wrapper.model.parameters(), lr=0.0005)
num_epochs = 15


start = time.time()
model_pipeline(DOTA_model_wrapper , optimizer, num_epochs, DOTA_train_dataset_loader,
               DOTA_val_dataset_loader, DOTA_test_dataset_loader, device,
               False, "Fovea Tracker")
end = time.time()
print(end - start)

In [ ]:
# save weights

In [ ]:
from google.colab import runtime
runtime.unassign()

## Fine-tune and test pre-trained model:

In [ ]:
backbone_fpn = ResNetCBAMCustomFPN(num_classes=3)
backbone_fpn.out_channels = 256

anchor_generator = AnchorGenerator(
    sizes=((32, 64, 128, 256),),
    aspect_ratios=((0.5, 1.0, 2.0),)
)

anchor_sizes = ((16,), (32,), (64,), (128,))
aspect_ratios = (((0.5,), (1.0,), (2.0,)),) * len(anchor_sizes)
anchor_generator = AnchorGenerator(anchor_sizes, aspect_ratios)

# Feature maps to perform RoI cropping.
# If backbone returns a Tensor, `featmap_names` is expected to
# be [0]. We can choose which feature maps to use.
roi_pooler = torchvision.ops.MultiScaleRoIAlign(
    featmap_names=["ll_feats",
                  "ml_1_feats",
                  "ml_2_feats",
                  "hl_feats"],
    output_size=7, #output size for pooled region
    sampling_ratio=2 #sampling ratio for ROIAlign
)

fine_tuned_model_wrapper = FasterRCNNWrapper(
    backbone=backbone_fpn,
    num_classes=17,
    rpn_anchor_generator=anchor_generator,
    box_roi_pool=roi_pooler
)
# BEFORE OR AFTER OVERRIDING HEADS ?
#load pre-trained weights and apply to chosen dataset
fine_tuned_model_wrapper.model.load_state_dict(torch.load(DOTA_pretrained_weights_path, weights_only=True))

num_classes=3 #2 classes +1 to account for background
fine_tuned_model_wrapper.override_prediction_heads(num_classes)

# NEED TO EXPERIMENT WITH FREEZING LAYERS HERE

In [ ]:
fine_tuned_model_wrapper.model.to(device)
# optimizer = torch.optim.Adam(cbam_model_wrapper.model.parameters(), lr=0.0005)
optimizer = torch.optim.Adam(fine_tuned_model_wrapper.model.parameters(), lr=0.00025)
num_epochs = 15
model_pipeline(fine_tuned_model_wrapper, optimizer, num_epochs, train_dataset_loader,
               val_dataset_loader, test_dataset_loader, device,
               True, "Custom Backbone Faster R-CNN Tracker")

In [ ]:
fine_tuned_DOTA_pretrained_weights_path = '/content/drive/MyDrive/Colab Notebooks/UoL FP/my_data_model_test/saved_models/fine_tuned_DOTA_model_weights.pth'
# torch.save(fine_tuned_model_wrapper.model.state_dict(), fine_tuned_DOTA_pretrained_weights_path)

## Faster R-CNN Hyperparameter tuning:

In [ ]:
!pip install optuna

In [ ]:
from torchvision.models.detection import FasterRCNN

def objective(trial):
  # set search space
  lr = trial.suggest_float("lr", 1e-5, 1e-1, log=True)
  anchor_sizes = trial.suggest_categorical("anchor_sizes", [
                                                              ((4,), (32,), (64,), (128,)),
                                                              ((6,), (8,), (64,), (128,)),
                                                              ((8,), (32,), (64,), (128,)),
                                                              ((8,), (16,), (32,), (64,))

                                                              ])
  optimizer_choice = trial.suggest_categorical("optimizer_choice", ["Adam",
                                                                    "AdamW", #reportedly good for computer vision tasks
                                                                    ])
  num_epochs = trial.suggest_int("num_epochs", 15, 20)

  # The following hyperparameter adjusts the RPN parameters:
  # rpn_fg_iou_thresh (float): minimum IoU between the anchor and the GT box so that they can be
  #     considered as positive during training of the RPN.

  # rpn_bg_iou_thresh (float): maximum IoU between the anchor and the GT box so that they can be
  #     considered as negative during training of the RPN.
  rpn_iou_thresh = trial.suggest_categorical("rpn_iou_thresh", [
                                                              [0.7, #rpn_fg_iou_thresh
                                                               0.3], #rpn_bg_iou_thresh
                                                              [0.6, #rpn_fg_iou_thresh
                                                               0.4],
                                                                ])
  # rpn_positive_fraction (float): proportion of positive anchors in a mini-batch during training of the RPN
  rpn_positive_fraction = trial.suggest_float("rpn_positive_fraction", 0.5, 0.8, step=0.1)





  # load data
  train_dataset = UAVImageDataset(data_dir_imgs="/content/drive/MyDrive/UoL Final Project/Project Data/official project data/2 classes - image size test/train/images",
                                df_path="/content/drive/MyDrive/UoL Final Project/Project Data/official project data/2 classes - image size test/train_pd.csv",
                                classes_start_at_zero=False)

  val_dataset = UAVImageDataset(data_dir_imgs="/content/drive/MyDrive/UoL Final Project/Project Data/official project data/2 classes - image size test/valid/images",
                                  df_path="/content/drive/MyDrive/UoL Final Project/Project Data/official project data/2 classes - image size test/valid_pd.csv",
                                classes_start_at_zero=False)

  # define data loaders
  train_dataset_loader = torch.utils.data.DataLoader(train_dataset,
                                                   collate_fn=lambda x: x,
                                                   batch_size=8,
                                                   shuffle=True,
                                                   pin_memory=True,
                                                  num_workers=12,
                                                   prefetch_factor=10
                                                   )


  val_dataset_loader = torch.utils.data.DataLoader(val_dataset, collate_fn=lambda x: x, batch_size=8,
                                                    num_workers=6)


  # define model
  backbone = FoveaBackbone(3)
  backbone.out_channels = 512

  aspect_ratios = (((0.5,), (1.0,), (2.0,)),) * len(anchor_sizes)
  anchor_generator = AnchorGenerator(anchor_sizes, aspect_ratios)

  roi_pooler = torchvision.ops.MultiScaleRoIAlign(
      # specify the feature maps to be used by the region proposal network
      featmap_names=["ll_feats",
                      "ml_1_feats",
                      "ml_2_feats",
                      "hl_feats"],
      output_size=7,
      sampling_ratio=2
  )

  model = FasterRCNN(
      backbone=backbone,
      num_classes=3,
      box_roi_pool=roi_pooler,
      # hyperparameters for the model
      rpn_anchor_generator=anchor_generator,
      rpn_fg_iou_thresh = rpn_iou_thresh[0],
      rpn_bg_iou_thresh = rpn_iou_thresh[1],
      rpn_positive_fraction = rpn_positive_fraction,
  )
  num_classes=3 #3 classes +1 to account for background
  in_features = model.roi_heads.box_predictor.cls_score.in_features
  model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)

  # load model weights (from pretraining on the dataset SODA)
  # SODA_weights_path = '/content/drive/MyDrive/Colab Notebooks/UoL FP/my_data_model_test/saved_models/SODA_pretrained_weights.pth'
  # model.load_state_dict(torch.load(SODA_weights_path, weights_only=True))

  device = torch.device("cuda:0")
  model.to(device)

  if optimizer_choice == "Adam":
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
  elif optimizer_choice == "AdamW":
    optimizer = torch.optim.AdamW(model.parameters(), lr=lr)

  model.train()
  for epoch in range(num_epochs):
      for batch in train_dataset_loader:
        batch_dict = format_batch(batch, model)
        del batch
        images, targets = batch_dict['images'], batch_dict['targets']
        loss_dict = model(images, targets)
        del images, targets, batch_dict
        training_loss = sum(v for v in loss_dict.values())
        del loss_dict
        # backward pass
        optimizer.zero_grad()
        training_loss.backward()
        optimizer.step()

  # calculate validation mAP metric
  with torch.no_grad():
    metric_map50 = MeanAveragePrecision(box_format="xyxy",iou_type="bbox", iou_thresholds=[0.5], class_metrics=True, extended_summary=True)
    #no iou_threshold leads to 50-95 coco mAP
    metric_map50_95 = MeanAveragePrecision(box_format="xyxy",iou_type="bbox", class_metrics=True, extended_summary=True)
    predictions = []
    ground_truths = []
    model.eval()
    for batch in val_dataset_loader:
      formatted_batch = format_batch(batch, model)
      images, targets = formatted_batch['images'], formatted_batch['targets']
      batch_predictions = model(images)
      predictions.extend(batch_predictions)
      ground_truths.extend(targets)

  # Update metric with predictions and respective ground truth
  metric_map50.update(predictions, ground_truths)
  metric_map50_95.update(predictions, ground_truths)
  # Compute the results
  map50_result = metric_map50.compute()
  map_50_95_result = metric_map50_95.compute()

  free_gpu_ram(train_dataset)
  free_gpu_ram(train_dataset_loader)
  free_gpu_ram(val_dataset)
  free_gpu_ram(val_dataset_loader)

  return map50_result['map'], map_50_95_result['map']


In [ ]:
import optuna

study = optuna.create_study(direction=['maximize', 'maximize'], #maximise both metrics returned by each trial
                            sampler=optuna.samplers.TPESampler(seed=42, multivariate=True), #Bayesian hyperparameter optimization
                            )

In [ ]:
study.optimize(objective, n_trials=20)

In [ ]:
best_params = study.best_params
best_params

### Analysis of hyperparameter tuning results:

In [ ]:
import matplotlib.pyplot as plt
optuna.visualization.plot_optimization_history(study)

In [ ]:
optuna.visualization.plot_parallel_coordinate(study)

In [ ]:
optuna.visualization.plot_slice(study, params=['learning_rate'])
# light blue represents early trials, dark blue represents trials towards end of search

In [ ]:
# hyperparameter importance
optuna.visualization.plot_param_importances(study)

## Test pre-trained model on test set from different source:

In [ ]:
alternate_test_dataset = UAVImageDataset(data_dir_imgs="/content/drive/MyDrive/UoL Final Project/Project Data/alternate testing sets/annotated_images_labels/images",
                                data_dir_targets="/content/drive/MyDrive/UoL Final Project/Project Data/alternate testing sets/annotated_images_labels/labels",
                                        classes_start_at_zero=False)

alternate_test_dataset_loader = torch.utils.data.DataLoader(alternate_test_dataset,
                                                   collate_fn=lambda x: x,
                                                   batch_size=16,
                                                   shuffle=True,
                                                   pin_memory=True,)

In [ ]:
backbone_fpn = ResNetCBAMCustomFPN(num_classes=3)
backbone_fpn.out_channels = 256

anchor_generator = AnchorGenerator(
    sizes=((32, 64, 128, 256),),
    aspect_ratios=((0.5, 1.0, 2.0),)
)

anchor_sizes = ((16,), (32,), (64,), (128,))
aspect_ratios = (((0.5,), (1.0,), (2.0,)),) * len(anchor_sizes)
anchor_generator = AnchorGenerator(anchor_sizes, aspect_ratios)

# Feature maps to perform RoI cropping.
# If backbone returns a Tensor, `featmap_names` is expected to
# be [0]. We can choose which feature maps to use.
roi_pooler = torchvision.ops.MultiScaleRoIAlign(
    featmap_names=["ll_feats",
                  "ml_1_feats",
                  "ml_2_feats",
                  "hl_feats"],
    output_size=7, #output size for pooled region
    sampling_ratio=2 #sampling ratio for ROIAlign
)

fine_tuned_model_wrapper = FasterRCNNWrapper(
    backbone=backbone_fpn,
    num_classes=3,
    rpn_anchor_generator=anchor_generator,
    box_roi_pool=roi_pooler
)
#load pre-trained weights and apply to chosen dataset
fine_tuned_model_wrapper.model.load_state_dict(torch.load(fine_tuned_DOTA_pretrained_weights_path, weights_only=True))

num_classes=3 #2 classes +1 to account for background
fine_tuned_model_wrapper.override_prediction_heads(num_classes)
fine_tuned_model_wrapper.model.to(device)
# test on alternate test set
fine_tuned_model_wrapper.test(alternate_test_dataset_loader)

## Faster R-CNN with SAHI

In [ ]:
# from torchmetrics.detection.mean_ap import MeanAveragePrecision
# from sahi.prediction import visualize_object_predictions
# import numpy
# import sahi

# # TEST !!!
# def sahi_inference_eval(detection_model: AutoDetectionModel,
#                         folder_path: str,
#                         slice_h_w: tuple[int, int],
#                         overlap_h_w: tuple[float, float],
#                         test_dataset_loader: torch.utils.data.dataloader,
#                         img_export_path: str) -> dict[str, float]:

#   """Executes Slicing Aided Hyper Inference (SAHI) for the given model

#     SAHI is implemented using the sahi pip library
#     Computes and returns the mAP50 and mAP75 values

#   Args:
#       detection_model: the model wrapped in a sahi AutoDetectionModel class
#       folder_path: the path to the folder of images
#       slice_h_w: the height and width of slices
#       overlap_h_w: the height and width of overlap ratio
#       test_dataset_loader: the PyTorch dataset loader
#       img_export_path: the path to the folder where resulting image predictions will be saved

#   Returns:
#       A dictionary of mAP values: mAP50, mAP75 and class wise mAP

#   Raises:
#       None
#   """

#   img_file_names = os.listdir(path=folder_path)

#   map_metric = MeanAveragePrecision(iou_type="bbox", iou_thresholds=[0.5, 0.75], class_metrics=True, extended_summary=True) #
#   all_predictions = []
#   all_ground_truths = []
#   i=0
#   for index, batch in enumerate(test_dataset_loader):
#     for img, targets, file_path in batch:
#       # get sliced predictions
#       img_predictions = get_sliced_prediction(
#         detection_model=detection_model,
#         image = file_path,
#         slice_height=slice_h_w[0],
#         slice_width=slice_h_w[1],
#         overlap_height_ratio=overlap_h_w[0],
#         overlap_width_ratio=overlap_h_w[1],
#         verbose=0
#       )

#       # visualise
#       img = cv2.imread(file_path, cv2.IMREAD_UNCHANGED)
#       img_converted = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
#       numpy_img= numpy.asarray(img_converted)
#       visualize_object_predictions(numpy_img,
#                                    object_prediction_list=img_predictions.object_prediction_list,
#                                    output_dir=img_export_path,
#                                    file_name=f"{i}",
#                                    hide_labels=1,
#                                    export_format="jpg"
#                                    )
#       i=i+1

#       # format predictions ready to calculate mAP metric
#       img_predictions = img_predictions.to_coco_annotations()
#       img_predictions = {
#           "boxes": torch.tensor([pred['bbox'] for pred in img_predictions]),
#           "scores": torch.tensor([pred['score'] for pred in img_predictions]),
#           "labels": torch.tensor([pred['category_id'] for pred in img_predictions]) #needs to be int ?
#       }
#       # convert bboxes annotations to same format as ground truth
#       for index, box in enumerate(img_predictions['boxes']):
#         x,y,w,h = box
#         img_predictions['boxes'][index][2] = x+w
#         img_predictions['boxes'][index][3] = y+h


#       boxes = img_predictions['boxes']
#       # apply nms
#       if len(boxes) > 1:
#         scores = img_predictions['scores']
#         indices_to_keep = torchvision.ops.nms(boxes, scores, 0.8) #0.8
#         #print(len(pred['boxes']))
#         #print(indices_to_keep)
#         pred = {
#             'boxes': img_predictions['boxes'][indices_to_keep],
#             'labels': img_predictions['labels'][indices_to_keep],
#             'scores': img_predictions['scores'][indices_to_keep],
#         }
#         all_predictions.append(pred)

#       else:
#         all_predictions.append(img_predictions)

#       all_ground_truths.append({"boxes": targets['boxes'], "labels": targets['labels']})

#       # visualisePredictions(0, test_dataset, [img_predictions])

#   map_metric.update(all_predictions, all_ground_truths)
#   map_metric = map_metric.compute()
#   return {'map50': map_metric['map_50'], 'map75': map_metric['map_75'], 'map_per_class': map_metric['map_per_class']}

In [ ]:
import os
# import required functions, classes
from sahi import AutoDetectionModel
from sahi.predict import get_sliced_prediction, predict, get_prediction
from sahi.utils.file import download_from_url
from sahi.utils.cv import read_image
from IPython.display import Image

detection_model = AutoDetectionModel.from_pretrained(
    model_type='torchvision',
    # model=cbam_model_wrapper.model,
    model = custom_model_wrapper.model,
    confidence_threshold=0.8,
    # image_size=640,
    device="cuda:0",
    load_at_init=True,
)

In [ ]:
source_image_dir = "/content/drive/MyDrive/UoL Final Project/Project Data/official project data/COCO_2_classes_test/images"
ground_truth = "/content/drive/MyDrive/UoL Final Project/Project Data/official project data/COCO_2_classes_test/_annotations.coco.json"
predictions = "/content/drive/MyDrive/Colab Notebooks/sahi_visuals/COCO_preds/pretrained_fasterrcnn.json"

slice_height = 256
slice_width = 256
overlap_height_ratio = 0.5
overlap_width_ratio = 0.5

sahi_inference_eval(detection_model=detection_model,
                    folder_path=source_image_dir,
                    slice_h_w=(slice_height, slice_width),
                    overlap_h_w=(overlap_height_ratio, overlap_width_ratio),
                    test_dataset_loader=test_dataset_loader,
                    img_export_path="/content/drive/MyDrive/Colab Notebooks/sahi_visuals/pretrained_fasterrcnn",
                    prediction_json_path=predictions,
                    ground_truth_json_path=ground_truth
                    )

## Appendix A: <a class="anchor" name="AppenA"></a>
Attempts to override FasterRCNN forward() method

In [ ]:
##***************************************TEST
from torchvision.models.detection import FasterRCNN

import warnings

class ModifiedFasterRCNN(FasterRCNN):
  def forward(self, images, targets=None):
        # type: (List[Tensor], Optional[List[Dict[str, Tensor]]]) -> Tuple[Dict[str, Tensor], List[Dict[str, Tensor]]]
        """
        Args:
            images (list[Tensor]): images to be processed
            targets (list[Dict[str, Tensor]]): ground-truth boxes present in the image (optional)

        Returns:
            result (list[BoxList] or dict[Tensor]): the output from the model.
                During training, it returns a dict[Tensor] which contains the losses.
                During testing, it returns list[BoxList] contains additional fields
                like `scores`, `labels` and `mask` (for Mask R-CNN models).

        """
        if self.training and targets is None:
            raise ValueError("In training mode, targets should be passed")
        if self.training:
            assert targets is not None
            for target in targets:
                boxes = target["boxes"]
                if isinstance(boxes, torch.Tensor):
                    if len(boxes.shape) != 2 or boxes.shape[-1] != 4:
                        raise ValueError(f"Expected target boxes to be a tensor of shape [N, 4], got {boxes.shape}.")
                else:
                    raise ValueError(f"Expected target boxes to be of type Tensor, got {type(boxes)}.")

        original_image_sizes: List[Tuple[int, int]] = []
        for img in images:
            val = img.shape[-2:]
            assert len(val) == 2
            original_image_sizes.append((val[0], val[1]))

        images, targets = self.transform(images, targets)

        # Check for degenerate boxes
        # TODO: Move this to a function
        if targets is not None:
            for target_idx, target in enumerate(targets):
                boxes = target["boxes"]
                degenerate_boxes = boxes[:, 2:] <= boxes[:, :2]
                if degenerate_boxes.any():
                    # print the first degenerate box
                    bb_idx = torch.where(degenerate_boxes.any(dim=1))[0][0]
                    degen_bb: List[float] = boxes[bb_idx].tolist()
                    raise ValueError(
                        "All bounding boxes should have positive height and width."
                        f" Found invalid box {degen_bb} for target at index {target_idx}."
                    )

        features = self.backbone(images.tensors)
        if isinstance(features, torch.Tensor):
            features = OrderedDict([("0", features)])
        proposals, proposal_losses = self.rpn(images, features, targets)
        detections, detector_losses = self.roi_heads(features, proposals, images.image_sizes, targets)
        detections = self.transform.postprocess(detections, images.image_sizes, original_image_sizes)  # type: ignore[operator]

        losses = {}
        losses.update(detector_losses)
        losses.update(proposal_losses)

        if torch.jit.is_scripting():
            if not self._has_warned:
                warnings.warn("RCNN always returns a (Losses, Detections) tuple in scripting")
                self._has_warned = True
            return losses, detections
        else:
            #MODIFIED THIS LINE
            return losses, detections

from torchvision.models.detection.backbone_utils import _resnet_fpn_extractor, _validate_trainable_layers, _mobilenet_extractor
from torchvision.ops import misc as misc_nn_ops
from torchvision.models.resnet import resnet50
from torchvision._internally_replaced_utils import load_state_dict_from_url
from torchvision.models.detection._utils import overwrite_eps

model_urls = {
    "fasterrcnn_resnet50_fpn_coco": "https://download.pytorch.org/models/fasterrcnn_resnet50_fpn_coco-258fb6c6.pth",
    "fasterrcnn_mobilenet_v3_large_320_fpn_coco": "https://download.pytorch.org/models/fasterrcnn_mobilenet_v3_large_320_fpn-907ea3f9.pth",
    "fasterrcnn_mobilenet_v3_large_fpn_coco": "https://download.pytorch.org/models/fasterrcnn_mobilenet_v3_large_fpn-fb6a3cc7.pth",
}

def modified_fasterrcnn_resnet50_fpn(
    pretrained=False, progress=True, num_classes=91, pretrained_backbone=True, trainable_backbone_layers=None, **kwargs
):
    """
    Constructs a Faster R-CNN model with a ResNet-50-FPN backbone.

    Reference: `"Faster R-CNN: Towards Real-Time Object Detection with
    Region Proposal Networks" <https://arxiv.org/abs/1506.01497>`_.

    The input to the model is expected to be a list of tensors, each of shape ``[C, H, W]``, one for each
    image, and should be in ``0-1`` range. Different images can have different sizes.

    The behavior of the model changes depending if it is in training or evaluation mode.

    During training, the model expects both the input tensors, as well as a targets (list of dictionary),
    containing:

        - boxes (``FloatTensor[N, 4]``): the ground-truth boxes in ``[x1, y1, x2, y2]`` format, with
          ``0 <= x1 < x2 <= W`` and ``0 <= y1 < y2 <= H``.
        - labels (``Int64Tensor[N]``): the class label for each ground-truth box

    The model returns a ``Dict[Tensor]`` during training, containing the classification and regression
    losses for both the RPN and the R-CNN.

    During inference, the model requires only the input tensors, and returns the post-processed
    predictions as a ``List[Dict[Tensor]]``, one for each input image. The fields of the ``Dict`` are as
    follows, where ``N`` is the number of detections:

        - boxes (``FloatTensor[N, 4]``): the predicted boxes in ``[x1, y1, x2, y2]`` format, with
          ``0 <= x1 < x2 <= W`` and ``0 <= y1 < y2 <= H``.
        - labels (``Int64Tensor[N]``): the predicted labels for each detection
        - scores (``Tensor[N]``): the scores of each detection

    For more details on the output, you may refer to :ref:`instance_seg_output`.

    Faster R-CNN is exportable to ONNX for a fixed batch size with inputs images of fixed size.

    Example::

        >>> model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=True)
        >>> # For training
        >>> images, boxes = torch.rand(4, 3, 600, 1200), torch.rand(4, 11, 4)
        >>> boxes[:, :, 2:4] = boxes[:, :, 0:2] + boxes[:, :, 2:4]
        >>> labels = torch.randint(1, 91, (4, 11))
        >>> images = list(image for image in images)
        >>> targets = []
        >>> for i in range(len(images)):
        >>>     d = {}
        >>>     d['boxes'] = boxes[i]
        >>>     d['labels'] = labels[i]
        >>>     targets.append(d)
        >>> output = model(images, targets)
        >>> # For inference
        >>> model.eval()
        >>> x = [torch.rand(3, 300, 400), torch.rand(3, 500, 400)]
        >>> predictions = model(x)
        >>>
        >>> # optionally, if you want to export the model to ONNX:
        >>> torch.onnx.export(model, x, "faster_rcnn.onnx", opset_version = 11)

    Args:
        pretrained (bool): If True, returns a model pre-trained on COCO train2017
        progress (bool): If True, displays a progress bar of the download to stderr
        num_classes (int): number of output classes of the model (including the background)
        pretrained_backbone (bool): If True, returns a model with backbone pre-trained on Imagenet
        trainable_backbone_layers (int): number of trainable (not frozen) resnet layers starting from final block.
            Valid values are between 0 and 5, with 5 meaning all backbone layers are trainable. If ``None`` is
            passed (the default) this value is set to 3.
    """
    trainable_backbone_layers = _validate_trainable_layers(
        pretrained or pretrained_backbone, trainable_backbone_layers, 5, 3
    )

    if pretrained:
        # no need to download the backbone if pretrained is set
        pretrained_backbone = False

    backbone = resnet50(pretrained=pretrained_backbone, progress=progress, norm_layer=misc_nn_ops.FrozenBatchNorm2d)
    backbone = _resnet_fpn_extractor(backbone, trainable_backbone_layers)
    #MODIFIED THE LINE BELOW
    model = ModifiedFasterRCNN(backbone, num_classes, **kwargs)

    if pretrained:
        state_dict = load_state_dict_from_url(model_urls["fasterrcnn_resnet50_fpn_coco"], progress=progress)
        model.load_state_dict(state_dict)
        overwrite_eps(model, 0.0)
    return model

In [ ]:
#TEST
model = modified_fasterrcnn_resnet50_fpn(weights=FasterRCNN_ResNet50_FPN_Weights.DEFAULT)

num_classes = 4  #+1 for background - there are 3 classes
in_features = model.roi_heads.box_predictor.cls_score.in_features
model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)

In [ ]:
# custom_model_state = custom_model.state_dict().__str__()
# model_state = model.state_dict().__str__()
# model_state == custom_model_state

## Appendix B: <a class="anchor" name="AppenB"></a>
Code used to check validity of Faster R-CNN implementation

In [ ]:
#model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=True) #- 0.1568 mAP50
#model = torchvision.models.detection.fasterrcnn_resnet50_fpn_v2(pretrained=True) - 0.1080 mAP50
#model = torchvision.models.detection.fasterrcnn_mobilenet_v3_large_fpn(pretrained=True) - 0.0434 mAP50
#model = torchvision.models.detection.fasterrcnn_mobilenet_v3_large_320_fpn(pretrained=True) - 0.0161 mAP50

#needs separate fine tuning process for this model
#model = torchvision.models.detection.ssd300_vgg16(pretrained=True)

# BASELINE CHECK
# from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
# from torchvision.models.detection import FasterRCNN_ResNet50_FPN_Weights

# model = torchvision.models.detection.fasterrcnn_resnet50_fpn(weights=FasterRCNN_ResNet50_FPN_Weights.DEFAULT)

# num_classes = 4  #+1 for background - there are 3 classes
# in_features = model.roi_heads.box_predictor.cls_score.in_features
# model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)

In [ ]:
# optimizer = torch.optim.SGD(model.parameters() , lr=0.0005, momentum=0.9, weight_decay=0.0005)
# num_epochs = 15
# model_pipeline(model, optimizer, num_epochs, train_dataset_loader,  val_dataset_loader, test_dataset_loader, device, False)

## Appendix C: Alternate version of Fovea backbone
took longer and had slightly worse performance:

In [ ]:
# the code in this cell is adapted from Kapil Sachdeva:
# https://www.youtube.com/watch?v=FKsgO0U7CUw&list=PLjRC2jqp3e7QNio7NJXRh_gvAdLV6_aZC&index=132
# the code has been modified so that it accomodates all 4 layers (feature maps) of the ResNetCBAM backbone (defined above)
from functools import partial
from torchvision.ops.misc import Conv2dNormActivation

class FPN3Layers(nn.Module):
  def __init__(self, in_channels: list[int], out_channels: int):
    super().__init__()

    LeakyRelu_Inplace = partial(
        nn.LeakyReLU,
        negative_slope=0.1,
        inplace=True,
    )

    ConvBlock = partial(Conv2dNormActivation, activation_layer=LeakyRelu_Inplace)

    self.hl_channel_reducer = ConvBlock(
        in_channels=in_channels[0],
        out_channels=out_channels,
        kernel_size=1,
    )

    self.ml_1_channel_reducer = ConvBlock(
        in_channels=in_channels[1],
        out_channels=out_channels,
        kernel_size=1,
    )

    self.ll_channel_reducer = ConvBlock(
        in_channels=in_channels[2],
        out_channels=out_channels,
        kernel_size=1,
    )

    self.ml_1_smoother = ConvBlock(
        in_channels=out_channels,
        out_channels=out_channels,
    )

    self.ll_smoother = ConvBlock(
        in_channels=out_channels,
        out_channels=out_channels,
    )

  def forward(self,
              hl_features:torch.Tensor,
              ml_1_features:torch.Tensor,
              ll_features:torch.Tensor):

    hl_channel_r = self.hl_channel_reducer(hl_features)
    del hl_features
    ml_channel_1_r = self.ml_1_channel_reducer(ml_1_features)
    del ml_1_features
    ll_channel_r = self.ll_channel_reducer(ll_features)
    del ll_features

    # addition 1
    upsample_hl = F.interpolate(
        hl_channel_r,
        size=[ml_channel_1_r.size(2), ml_channel_1_r.size(3)],
        mode="nearest",
    )

    fused_hl_ml_1 = upsample_hl + ml_channel_1_r
    smoothed_ml_1_features = self.ml_1_smoother(fused_hl_ml_1)
    # memory management
    del ml_channel_1_r, upsample_hl, fused_hl_ml_1

    # addition 3
    upsample_ml_1 = F.interpolate(
        smoothed_ml_1_features,
        size=[ll_channel_r.size(2), ll_channel_r.size(3)],
        mode="nearest",
    )
    fused_ml_ll = upsample_ml_1 + ll_channel_r
    smoothed_ll_features = self.ll_smoother(fused_ml_ll)
    # memory management
    del ll_channel_r, fused_ml_ll, upsample_ml_1


    out = [hl_channel_r,
           smoothed_ml_1_features,
           smoothed_ll_features]
    return out

In [ ]:
import torch.nn as nn

class ShallowFovea(nn.Module):
  def __init__(self, num_channels: int, out_channels: int):
    super().__init__()

    LeakyRelu_Inplace = partial(
        nn.LeakyReLU,
        negative_slope=0.1,
        inplace=True,
    )

    ConvBlock = partial(Conv2dNormActivation, activation_layer=LeakyRelu_Inplace)
    channel_increment_halves = out_channels-num_channels // 2
    channel_increment_quarter = out_channels-num_channels // 4

    self.branch_1 = nn.Sequential(
        ConvBlock(in_channels=num_channels, out_channels=num_channels+channel_increment_halves, kernel_size=1),
        ConvBlock(in_channels=num_channels+channel_increment_halves, out_channels=num_channels+(channel_increment_halves*2), kernel_size=7),
    )

    self.branch_2 = nn.Sequential(
        ConvBlock(in_channels=num_channels, out_channels=num_channels+channel_increment_quarter, kernel_size=1),
        ConvBlock(in_channels=num_channels+channel_increment_quarter, out_channels=num_channels+(channel_increment_quarter*2), kernel_size=(1,7)),
        ConvBlock(in_channels=num_channels+(channel_increment_quarter*2), out_channels=num_channels+(channel_increment_quarter*3), kernel_size=(7,1)),
        ConvBlock(in_channels=num_channels+(channel_increment_quarter*3), out_channels=num_channels+(channel_increment_quarter*4), kernel_size=7, dilation=3),
    )

    self.branch_3 = nn.Sequential(
        ConvBlock(in_channels=num_channels, out_channels=num_channels+channel_increment_quarter, kernel_size=1),
        ConvBlock(in_channels=num_channels+channel_increment_quarter, out_channels=num_channels+(channel_increment_quarter*2), kernel_size=(3,1)),
        ConvBlock(in_channels=num_channels+(channel_increment_quarter*2), out_channels=num_channels+(channel_increment_quarter*3), kernel_size=(1,3)),
        ConvBlock(in_channels=num_channels+(channel_increment_quarter*3), out_channels=num_channels+(channel_increment_quarter*4), kernel_size=7, dilation=3),
    )

    self.branch_4 = nn.Sequential(
        ConvBlock(in_channels=num_channels, out_channels=out_channels, kernel_size=1),
    )

    self.downsampler = nn.LazyConv2d(out_channels=out_channels, kernel_size=3)


  def forward(self, x):
    branch_1 = self.branch_1(x)
    branch_2 = self.branch_2(x)
    branch_3 = self.branch_3(x)
    branch_4 = self.branch_4(x)

    upsampled_branch_1 = F.interpolate(
        branch_1,
        size=[branch_3.size(2), branch_3.size(3)],
        mode="bilinear",
    )
    del branch_1 #remove intermediate calculation from GPU RAM clear
    upsampled_branch_2 = F.interpolate(
        branch_2,
        size=[branch_3.size(2), branch_3.size(3)],
        mode="bilinear",
    )
    del branch_2 #remove intermediate calculation from GPU RAM clear

    branch_concat = torch.cat( (upsampled_branch_1,
                                upsampled_branch_2,
                                branch_3), dim=1 ) #concat on first dimension to allow for different number of channels
    del upsampled_branch_1, upsampled_branch_2, branch_3


    # prep ready for element wise multiplication
    downsampled_branch_concat = self.downsampler(branch_concat)
    del branch_concat #remove intermediate calculation from GPU RAM clear

    resized_branch_concat = F.interpolate(
        downsampled_branch_concat,
        size=[branch_4.size(2), branch_4.size(3)],
        mode="bilinear",
    )
    del downsampled_branch_concat
    result = branch_4 * resized_branch_concat
    del branch_4, resized_branch_concat
    return result

In [ ]:
import torch.nn as nn

class DeepFovea(nn.Module):
  def __init__(self, num_channels: int, out_channels: int):
    super().__init__()

    LeakyRelu_Inplace = partial(
        nn.LeakyReLU,
        negative_slope=0.1,
        inplace=True,
    )

    ConvBlock = partial(Conv2dNormActivation, activation_layer=LeakyRelu_Inplace)
    channel_increment_halves = out_channels-num_channels // 2
    channel_increment_quarter = out_channels-num_channels // 4

    self.branch_1 = nn.Sequential(
        ConvBlock(in_channels=num_channels, out_channels=num_channels+channel_increment_halves, kernel_size=1),
        ConvBlock(in_channels=num_channels+channel_increment_halves, out_channels=num_channels+(channel_increment_halves*2), kernel_size=3),
    )

    self.branch_2 = nn.Sequential(
        ConvBlock(in_channels=num_channels, out_channels=num_channels+channel_increment_quarter, kernel_size=1),
        ConvBlock(in_channels=num_channels+channel_increment_quarter, out_channels=num_channels+(channel_increment_quarter*2), kernel_size=(1,3)),
        ConvBlock(in_channels=num_channels+(channel_increment_quarter*2), out_channels=num_channels+(channel_increment_quarter*3), kernel_size=(3,1)),
        ConvBlock(in_channels=num_channels+(channel_increment_quarter*3), out_channels=num_channels+(channel_increment_quarter*4), kernel_size=3, dilation=3),
    )

    self.branch_3 = nn.Sequential(
        ConvBlock(in_channels=num_channels, out_channels=num_channels+channel_increment_quarter, kernel_size=1),
        ConvBlock(in_channels=num_channels+channel_increment_quarter, out_channels=num_channels+(channel_increment_quarter*2), kernel_size=(3,1)),
        ConvBlock(in_channels=num_channels+(channel_increment_quarter*2), out_channels=num_channels+(channel_increment_quarter*3), kernel_size=(1,3)),
        ConvBlock(in_channels=num_channels+(channel_increment_quarter*3), out_channels=num_channels+(channel_increment_quarter*4), kernel_size=3, dilation=3),
    )

    self.branch_4 = nn.Sequential(
        ConvBlock(in_channels=num_channels, out_channels=out_channels, kernel_size=1),
    )

    self.downsampler = nn.LazyConv2d(out_channels=out_channels, kernel_size=3)


  def forward(self, x):
    branch_1 = self.branch_1(x)
    branch_2 = self.branch_2(x)
    branch_3 = self.branch_3(x)
    branch_4 = self.branch_4(x)

    upsampled_branch_1 = F.interpolate(
        branch_1,
        size=[branch_3.size(2), branch_3.size(3)],
        mode="bilinear",
    )
    del branch_1 #remove intermediate calculation from GPU RAM clear

    upsampled_branch_2 = F.interpolate(
        branch_2,
        size=[branch_3.size(2), branch_3.size(3)],
        mode="bilinear",
    )
    del branch_2 #remove intermediate calculation from GPU RAM clear

    branch_concat = torch.cat( (upsampled_branch_1,
                                upsampled_branch_2,
                                branch_3), dim=1 ) #concat on first dimension to allow for different number of channels

    del upsampled_branch_1, upsampled_branch_2, branch_3

    # prep ready for element wise multiplication
    downsampled_branch_concat = self.downsampler(branch_concat)
    del branch_concat  #remove intermediate calculation from GPU RAM clear

    resized_branch_concat = F.interpolate(
        downsampled_branch_concat,
        size=[branch_4.size(2), branch_4.size(3)],
        mode="bilinear",
    )
    del downsampled_branch_concat

    result = branch_4 * resized_branch_concat
    del branch_4, resized_branch_concat
    return result

In [ ]:
# TO DO:
# - experiment with residual connections


class FoveaBackbone(nn.Module):
  def __init__(self, in_channels):
    super().__init__()

    LeakyRelu_Inplace = partial(
        nn.LeakyReLU,
        negative_slope=0.1,
        inplace=True,
    )

    ConvBlock = partial(Conv2dNormActivation, activation_layer=LeakyRelu_Inplace)

    #stride not mention in EVM paper but needed as imgs so large
    self.block_1 = nn.Sequential(
        ConvBlock(in_channels=in_channels, out_channels=32, kernel_size=3, stride=2),
        nn.MaxPool2d(kernel_size=2)
    )

    self.shallow_fovea_1 = ShallowFovea(num_channels=32, out_channels=64)
    self.deep_fovea_1 = DeepFovea(num_channels=32, out_channels=64)

    self.shallow_fovea_2 = ShallowFovea(num_channels=64, out_channels=128)
    self.deep_fovea_2 = DeepFovea(num_channels=64, out_channels=128)

    self.shallow_fovea_3 = ShallowFovea(num_channels=128, out_channels=256)
    self.deep_fovea_3 = DeepFovea(num_channels=128, out_channels=256)

    # self.shallow_fovea_4 = ShallowFovea(num_channels=256, out_channels=512)
    # self.deep_fovea_4 = DeepFovea(num_channels=256, out_channels=512)

    self.max_pooler = nn.MaxPool2d(kernel_size=2)
    self.fpn = FPN3Layers(in_channels=[64, 128, 256], out_channels=256)

    self.cbam_scaler_1 = nn.Parameter(torch.zeros(1))
    self.cbam_1 = ScaledCBAM(in_channels=64, scaling_parameter=self.cbam_scaler_1)
    self.cbam_scaler_2 = nn.Parameter(torch.zeros(1))
    self.cbam_2 = ScaledCBAM(in_channels=128, scaling_parameter=self.cbam_scaler_2)
    self.cbam_scaler_3 = nn.Parameter(torch.zeros(1))
    self.cbam_3 = ScaledCBAM(in_channels=256, scaling_parameter=self.cbam_scaler_3)
    # self.cbam_scaler_4 = nn.Parameter(torch.zeros(1))
    # self.cbam_4 = ScaledCBAM(in_channels=512, scaling_parameter=self.cbam_scaler_4)

  def forward(self, x):
    x = self.block_1(x)

    shallow_feats_1 = self.shallow_fovea_1(x)
    deep_feats_1 = self.deep_fovea_1(x)
    stage_1_results = shallow_feats_1 + deep_feats_1
    stage_1_results_pooled = self.max_pooler(stage_1_results) #reduce computational load for successive layers
    stage_1_results_w_cbam = self.cbam_1(stage_1_results)
    # memory management
    del shallow_feats_1, deep_feats_1, stage_1_results
    # upsample and add residual connection - or downsample ?
    # try one with upsample and one with downsample
    # print(x.size())
    # print(stage_1_results_w_cbam.size())
    # stop
    # torch.Size([8, 32, 200, 272])
    # torch.Size([8, 64, 200, 272])

    shallow_feats_2 = self.shallow_fovea_2(stage_1_results_pooled)
    deep_feats_2 = self.deep_fovea_2(stage_1_results_pooled)
    stage_2_results = shallow_feats_2 + deep_feats_2
    stage_2_results_pooled = self.max_pooler(stage_2_results) #reduce computational load for successive layers
    stage_2_results_w_cbam = self.cbam_2(stage_2_results)
    # memory management
    del shallow_feats_2, deep_feats_2, stage_2_results, stage_1_results_pooled,

    shallow_feats_3 = self.shallow_fovea_3(stage_2_results_pooled)
    deep_feats_3 = self.deep_fovea_3(stage_2_results_pooled)
    stage_3_results = shallow_feats_3 + deep_feats_3
    stage_3_results_pooled = self.max_pooler(stage_3_results) #reduce computational load for successive layers
    stage_3_results_w_cbam = self.cbam_3(stage_3_results)
    # memory management
    del shallow_feats_3, deep_feats_3, stage_3_results, stage_2_results_pooled,

    # shallow_feats_4 = self.shallow_fovea_4(stage_3_results_pooled)
    # deep_feats_4 = self.deep_fovea_4(stage_3_results_pooled)
    # stage_4_results = shallow_feats_4 + deep_feats_4
    # stage_4_results_w_cbam = self.cbam_4(stage_4_results)
    # # memory management
    # del shallow_feats_4, deep_feats_4, stage_4_results, stage_3_results_pooled,

    feat_maps = self.fpn(hl_features = stage_1_results_w_cbam,
                        ml_1_features= stage_2_results_w_cbam,
                        ll_features = stage_3_results_w_cbam, )

    del stage_1_results_w_cbam, stage_2_results_w_cbam, stage_3_results_w_cbam, #stage_4_results_w_cbam

    return OrderedDict([(k, v) for k, v in zip(["hl_feats",
                                                "ml_1_feats",
                                                # "ml_2_feats",
                                                "ll_feats"], feat_maps)])

In [ ]:
import torchvision
from torchvision.models.detection import FasterRCNN
from torchvision.models.detection.rpn import AnchorGenerator

backbone = FoveaBackbone(3)
backbone.out_channels = 256

anchor_sizes = ((8,), (44,), (128,))  #16, 32, 64, 128
aspect_ratios = (((0.5,), (1.0,), (2.0,)),) * len(anchor_sizes)
anchor_generator = AnchorGenerator(anchor_sizes, aspect_ratios)

# Feature maps to perform RoI cropping.
# If backbone returns a Tensor, `featmap_names` is expected to
# be [0]. We can choose which feature maps to use.
roi_pooler = torchvision.ops.MultiScaleRoIAlign(
    featmap_names=["hl_feats",
                    "ml_1_feats",
                    # "ml_2_feats",
                    "ll_feats"],
    output_size=7,
    sampling_ratio=2
)

fovea_model_wrapper = FasterRCNNWrapper(
    backbone=backbone,
    num_classes=3,
    rpn_anchor_generator=anchor_generator,
    box_roi_pool=roi_pooler
)
num_classes=3 #3 classes +1 to account for background
fovea_model_wrapper.override_prediction_heads(num_classes)

In [ ]:
fovea_model_wrapper.model.to(device)
# optimizer = torch.optim.SGD(model.model.parameters() , lr=0.0005, momentum=0.9, weight_decay=0.0005)
optimizer = torch.optim.Adam(fovea_model_wrapper.model.parameters(), lr=0.0005)
num_epochs = 15
model_pipeline(fovea_model_wrapper , optimizer, num_epochs, train_dataset_loader,
               val_dataset_loader, test_dataset_loader, device,
               True, "Fovea Tracker")